In [2]:
import pandas as pd
import pandas.tseries.offsets as offsets
from IPython.display import display
import jpholiday
import datetime
import numpy as np
from sklearn import preprocessing
import pandas_profiling as pdp

In [3]:

pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 10000)

In [4]:
tourism_train = pd.read_csv("../data/train.tsv",delimiter="\t")
print(tourism_train.shape)
tourism_train.head(20)


(5848, 3)


,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
1,2015-01-01,十和田八幡平国立公園,34757
2,2015-01-01,日光国立公園,29714
3,2015-01-01,伊勢志摩国立公園,42652
4,2015-01-01,大山隠岐国立公園,3637
5,2015-01-01,阿蘇くじゅう国立公園,1369
6,2015-01-01,霧島錦江湾国立公園,35352
7,2015-01-01,慶良間諸島国立公園,151
8,2015-01-02,阿寒摩周国立公園,11153
9,2015-01-02,十和田八幡平国立公園,33795


In [6]:
#観光及び国立公園に関連する語句が含まれる投稿の2015/01/01～2017/12/30までの日別件数（ブログ・twitter・掲示板）データ。
sns = pd.read_csv("../data/hotlink.tsv",delimiter="\t")
pdp.ProfileReport(sns)

Number of variables,4
Number of observations,144540
Total Missing (%),0.0%
Total size in memory,4.4 MiB
Average record size in memory,32.0 B
Numeric,1
Categorical,3
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [8]:
sns["keyword"].unique()

array(['国立公園', 'アオウミガメ', '阿波連岬園地', '鬼怒川', '稲崎園地', '東照宮', 'ミヤマキリシマ',
       'ノカイドウ', '栗野八幡地獄', '開聞岳', 'あか牛', '日光の社寺', 'キスミレ', 'ザトウクジラ',
       '慶良間諸島国立公園', '霧島錦江湾国立公園', '阿蘇くじゅう国立公園', '大山隠岐国立公園', '華厳の滝',
       '伊勢志摩国立公園', '十和田八幡平国立公園', '阿寒摩周国立公園', '日光国立公園', '小松地獄', 'リアス海岸',
       '伊勢神宮', '雌阿寒岳', 'モリアオガエル', 'コマクサ', '岩手山', '奥入瀬渓流', '双湖台', '八甲田山',
       '屈斜路湖', 'オンネトー', 'アカウミガメ', '鶴見岳', 'ダイセンキャラボク', '中禅寺湖', 'オキサンショウウオ',
       '出雲大社', '隠岐自然回帰の森', '浄土ヶ浦', 'ハマユウ'], dtype=object)

In [5]:
#日光国立公園区画内のSNSロケーション解析データ投稿数データ。
locsns = pd.read_csv("../data/nightley.tsv",delimiter="\t")
print(locsns.shape)
locsns.head()


(730, 3)


,datetime,Japan_count,Foreign_count
0,2016-01-01,28,0
1,2016-01-02,66,1
2,2016-01-03,75,6
3,2016-01-04,50,1
4,2016-01-05,25,0


In [6]:
visitors = pd.read_csv("../data/colopl.tsv",delimiter="\t")
print(visitors.shape)
visitors.head()

(1701, 5)


,year,month,park,country_jp,count
0,2016,1,阿寒摩周国立公園,アメリカ合衆国,1-9
1,2016,1,阿寒摩周国立公園,シンガポール,1-9
2,2016,1,阿寒摩周国立公園,マレーシア,1-9
3,2016,1,阿寒摩周国立公園,中国,1-9
4,2016,1,阿寒摩周国立公園,台湾,19


In [161]:
#国立公園周辺12地点の気象観測データ。
weather = pd.read_csv("../data/weather.tsv",delimiter="\t")
print(weather.shape)
weather.tail(100)

(13140, 24)


,年月日,地点,平均気温(℃),最高気温(℃),最低気温(℃),降水量の合計(mm),10分間降水量の最大(mm),日照時間(時間),合計全天日射量(MJ/㎡),最深積雪(cm),降雪量合計(cm),平均風速(m/s),最大風速(m/s),最大瞬間風速(m/s),最多風向(16方位),平均蒸気圧(hPa),平均現地気圧(hPa),平均湿度(%),平均海面気圧(hPa),最小相対湿度(%),最低海面気圧(hPa),平均雲量(10分比),天気概況(昼:06時~18時),天気概況(夜:18時~翌日06時)
13040,2017/9/22,鹿角,15.9,25.1,9.1,0.0,0.0,9.4,NaN,0.0,0.0,0.7,2.6,4.7,北北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13041,2017/9/23,鹿角,16.9,24.2,11.4,2.0,2.0,1.3,NaN,0.0,0.0,0.7,2.4,4.7,北北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13042,2017/9/24,鹿角,17.6,24.0,12.7,0.0,0.0,6.4,NaN,0.0,0.0,1.1,3.6,6.4,西北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13043,2017/9/25,鹿角,16.3,22.9,11.1,0.0,0.0,5.7,NaN,0.0,0.0,1.1,4.0,6.7,西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044,2017/9/26,鹿角,17.1,23.5,11.3,0.0,0.0,10.1,NaN,0.0,0.0,1.2,4.2,7.1,西北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13045,2017/9/27,鹿角,16.8,23.4,11.2,2.0,1.0,3.3,NaN,0.0,0.0,0.8,4.9,8.4,西北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13046,2017/9/28,鹿角,12.0,16.6,6.8,38.5,2.0,0.0,NaN,0.0,0.0,1.0,5.1,7.6,西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13047,2017/9/29,鹿角,10.1,18.3,4.7,6.0,2.0,3.8,NaN,0.0,0.0,0.7,5.4,8.8,南,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13048,2017/9/30,鹿角,9.5,14.6,6.3,6.5,1.0,1.8,NaN,0.0,0.0,0.6,2.1,5.6,北東,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13049,2017/10/1,鹿角,12.5,21.1,5.6,0.0,0.0,5.5,NaN,0.0,0.0,0.7,3.0,4.8,北北西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#防災科学技術研究所 ⼤⼭鏡ヶ成積雪気象観測施設における積雪気象観測データ。
snow = pd.read_csv("../data/nied_oyama.tsv",delimiter="\t")
print(snow.shape)
snow.head()

(12780, 5)


,日時,積雪深(cm),積雪深重量(kg m^-2),気温(℃),降水量(mm h-1)
0,2015-01-01 00:00:00,99.3,305.1,-4.8,1.0
1,2015-01-01 01:00:00,101.9,308.3,-4.8,1.0
2,2015-01-01 02:00:00,102.5,310.3,-5.3,0.0
3,2015-01-01 03:00:00,101.8,310.6,-5.5,0.0
4,2015-01-01 04:00:00,101.9,311.1,-6.1,0.5


In [9]:
#各国立公園付近エリア単位の「平均人口」の集計値データ
year = 2017
month = 3
population_dic = {}
population_dic[(year, month)] = pd.read_csv("../data/month_time_mesh100m_"+str(year)+"0"+str(month)+".tsv",delimiter="\t")
print(population_dic[(year, month)].shape)
population_dic[(year, month)].head(30)

(265735, 7)


,park,mesh100mid,year,month,dayflag,hour,population
0,伊勢志摩国立公園,51362395334,2017,3,0,16,4
1,伊勢志摩国立公園,51362395334,2017,3,1,9,11
2,伊勢志摩国立公園,51362395334,2017,3,2,9,8
3,伊勢志摩国立公園,51362395334,2017,3,2,16,1
4,伊勢志摩国立公園,51362396400,2017,3,1,0,47
5,伊勢志摩国立公園,51362396400,2017,3,1,1,97
6,伊勢志摩国立公園,51362396400,2017,3,1,2,97
7,伊勢志摩国立公園,51362396400,2017,3,1,3,97
8,伊勢志摩国立公園,51362396400,2017,3,1,4,97
9,伊勢志摩国立公園,51362396400,2017,3,1,5,97


In [10]:
isesima = tourism_train.loc[tourism_train["park"] == "伊勢志摩国立公園",]
print(isesima.shape)
isesima.to_csv("../data/isesima.csv",index=False)
isesima.head()


(731, 3)


,datetime,park,visitors
3,2015-01-01,伊勢志摩国立公園,42652
11,2015-01-02,伊勢志摩国立公園,47035
19,2015-01-03,伊勢志摩国立公園,45368
27,2015-01-04,伊勢志摩国立公園,34960
35,2015-01-05,伊勢志摩国立公園,17740


In [11]:
towada = tourism_train.loc[tourism_train["park"] == "十和田八幡平国立公園",]
print(towada.shape)
towada.to_csv("../data/towada.csv",index=False)
towada.head()


(731, 3)


,datetime,park,visitors
1,2015-01-01,十和田八幡平国立公園,34757
9,2015-01-02,十和田八幡平国立公園,33795
17,2015-01-03,十和田八幡平国立公園,17114
25,2015-01-04,十和田八幡平国立公園,9306
33,2015-01-05,十和田八幡平国立公園,4732


In [12]:
nikko = tourism_train.loc[tourism_train["park"] == "日光国立公園",]
print(nikko.shape)
nikko.to_csv("../data/nikko.csv",index=False)
nikko.head()


(731, 3)


,datetime,park,visitors
2,2015-01-01,日光国立公園,29714
10,2015-01-02,日光国立公園,27011
18,2015-01-03,日光国立公園,22247
26,2015-01-04,日光国立公園,12236
34,2015-01-05,日光国立公園,6571


In [13]:
akan = tourism_train.loc[tourism_train["park"] == "阿寒摩周国立公園",]
print(akan.shape)
akan.to_csv("../data/akan.csv",index=False)
akan.head()


(731, 3)


,datetime,park,visitors
0,2015-01-01,阿寒摩周国立公園,11028
8,2015-01-02,阿寒摩周国立公園,11153
16,2015-01-03,阿寒摩周国立公園,12343
24,2015-01-04,阿寒摩周国立公園,6732
32,2015-01-05,阿寒摩周国立公園,4877


In [14]:
ooyama = tourism_train.loc[tourism_train["park"] == "大山隠岐国立公園",]
print(ooyama.shape)
ooyama.to_csv("../data/ooyama.csv",index=False)
ooyama.head()


(731, 3)


,datetime,park,visitors
4,2015-01-01,大山隠岐国立公園,3637
12,2015-01-02,大山隠岐国立公園,3244
20,2015-01-03,大山隠岐国立公園,1927
28,2015-01-04,大山隠岐国立公園,880
36,2015-01-05,大山隠岐国立公園,135


In [15]:
aso = tourism_train.loc[tourism_train["park"] == "阿蘇くじゅう国立公園",]
print(aso.shape)
aso.to_csv("../data/aso.csv",index=False)
aso.head()


(731, 3)


,datetime,park,visitors
5,2015-01-01,阿蘇くじゅう国立公園,1369
13,2015-01-02,阿蘇くじゅう国立公園,1643
21,2015-01-03,阿蘇くじゅう国立公園,967
29,2015-01-04,阿蘇くじゅう国立公園,324
37,2015-01-05,阿蘇くじゅう国立公園,100


In [16]:
kirisima = tourism_train.loc[tourism_train["park"] == "霧島錦江湾国立公園",]
print(kirisima.shape)
kirisima.to_csv("../data/kirisima.csv",index=False)
kirisima.head()


(731, 3)


,datetime,park,visitors
6,2015-01-01,霧島錦江湾国立公園,35352
14,2015-01-02,霧島錦江湾国立公園,29856
22,2015-01-03,霧島錦江湾国立公園,27027
30,2015-01-04,霧島錦江湾国立公園,12993
38,2015-01-05,霧島錦江湾国立公園,8112


In [17]:
kerama = tourism_train.loc[tourism_train["park"] == "慶良間諸島国立公園",]
print(kerama.shape)
kerama.to_csv("../data/kerama.csv",index=False)
kerama.head()

(731, 3)


,datetime,park,visitors
7,2015-01-01,慶良間諸島国立公園,151
15,2015-01-02,慶良間諸島国立公園,657
23,2015-01-03,慶良間諸島国立公園,126
31,2015-01-04,慶良間諸島国立公園,75
39,2015-01-05,慶良間諸島国立公園,0


In [18]:
lag = 8
target = "aso"

data = pd.DataFrame()
data[target] = pd.read_csv("../data/"+target+".csv")["visitors"]
data.index = pd.to_datetime(pd.read_csv("../data/"+target+".csv")["datetime"])
data.head(20)

,aso
datetime,
2015-01-01,1369
2015-01-02,1643
2015-01-03,967
2015-01-04,324
2015-01-05,100
2015-01-06,0
2015-01-07,25
2015-01-08,0
2015-01-09,50


In [19]:
feature = pd.DataFrame()
for l in list(range(1,lag+1)):
    feature.loc[:,target + "lag"+str(l)] = data[target].shift(-(lag-l))
feature.loc[:,"y"] = data[target].shift(-lag)
feature.index = feature.index + offsets.Day(lag)
feature

,asolag1,asolag2,asolag3,asolag4,asolag5,asolag6,asolag7,asolag8,y
datetime,,,,,,,,,
2015-01-09,0.0,25.0,0.0,100.0,324.0,967.0,1643.0,1369,50.0
2015-01-10,50.0,0.0,25.0,0.0,100.0,324.0,967.0,1643,448.0
2015-01-11,448.0,50.0,0.0,25.0,0.0,100.0,324.0,967,174.0
2015-01-12,174.0,448.0,50.0,0.0,25.0,0.0,100.0,324,0.0
2015-01-13,0.0,174.0,448.0,50.0,0.0,25.0,0.0,100,50.0
2015-01-14,50.0,0.0,174.0,448.0,50.0,0.0,25.0,0,50.0
2015-01-15,50.0,50.0,0.0,174.0,448.0,50.0,0.0,25,100.0
2015-01-16,100.0,50.0,50.0,0.0,174.0,448.0,50.0,0,446.0
2015-01-17,446.0,100.0,50.0,50.0,0.0,174.0,448.0,50,347.0


In [20]:
feature.to_csv("../data/"+target+"l"+str(lag)+".csv")

# visitor lag

In [103]:
lag = 7
targets = ["aso","akan","isesima","ooyama","kirisima","kerama","towada","nikko"]

for target in targets:
    data = pd.DataFrame()
    data[target] = pd.read_csv("../data/"+target+".csv")["visitors"]
    data.index = pd.to_datetime(pd.read_csv("../data/"+target+".csv")["datetime"])
    
    feature = pd.DataFrame()
    for l in list(range(1,lag+1)):
        feature.loc[:,target + "lag"+str(l)] = data[target].shift(-(lag-l))
    feature.loc[:,"y"] = data[target].shift(-lag)
    feature["date"]= feature.index + offsets.Day(lag)
    feature.index = feature.index + offsets.Day(lag)
    display(feature.head())
    feature.to_csv("../data/"+target+"_visitor_lag"+str(lag)+".csv")

,asolag1,asolag2,asolag3,asolag4,asolag5,asolag6,asolag7,y,date
datetime,,,,,,,,,
2015-01-08,25.0,0.0,100.0,324.0,967.0,1643.0,1369,0.0,2015-01-08
2015-01-09,0.0,25.0,0.0,100.0,324.0,967.0,1643,50.0,2015-01-09
2015-01-10,50.0,0.0,25.0,0.0,100.0,324.0,967,448.0,2015-01-10
2015-01-11,448.0,50.0,0.0,25.0,0.0,100.0,324,174.0,2015-01-11
2015-01-12,174.0,448.0,50.0,0.0,25.0,0.0,100,0.0,2015-01-12


,akanlag1,akanlag2,akanlag3,akanlag4,akanlag5,akanlag6,akanlag7,y,date
datetime,,,,,,,,,
2015-01-08,4569.0,4162.0,4877.0,6732.0,12343.0,11153.0,11028,2715.0,2015-01-08
2015-01-09,2715.0,4569.0,4162.0,4877.0,6732.0,12343.0,11153,2258.0,2015-01-09
2015-01-10,2258.0,2715.0,4569.0,4162.0,4877.0,6732.0,12343,4273.0,2015-01-10
2015-01-11,4273.0,2258.0,2715.0,4569.0,4162.0,4877.0,6732,4273.0,2015-01-11
2015-01-12,4273.0,4273.0,2258.0,2715.0,4569.0,4162.0,4877,2686.0,2015-01-12


,isesimalag1,isesimalag2,isesimalag3,isesimalag4,isesimalag5,isesimalag6,isesimalag7,y,date
datetime,,,,,,,,,
2015-01-08,4612.0,9118.0,17740.0,34960.0,45368.0,47035.0,42652,3489.0,2015-01-08
2015-01-09,3489.0,4612.0,9118.0,17740.0,34960.0,45368.0,47035,5327.0,2015-01-09
2015-01-10,5327.0,3489.0,4612.0,9118.0,17740.0,34960.0,45368,17676.0,2015-01-10
2015-01-11,17676.0,5327.0,3489.0,4612.0,9118.0,17740.0,34960,21679.0,2015-01-11
2015-01-12,21679.0,17676.0,5327.0,3489.0,4612.0,9118.0,17740,10215.0,2015-01-12


,ooyamalag1,ooyamalag2,ooyamalag3,ooyamalag4,ooyamalag5,ooyamalag6,ooyamalag7,y,date
datetime,,,,,,,,,
2015-01-08,542.0,141.0,135.0,880.0,1927.0,3244.0,3637,270.0,2015-01-08
2015-01-09,270.0,542.0,141.0,135.0,880.0,1927.0,3244,88.0,2015-01-09
2015-01-10,88.0,270.0,542.0,141.0,135.0,880.0,1927,384.0,2015-01-10
2015-01-11,384.0,88.0,270.0,542.0,141.0,135.0,880,613.0,2015-01-11
2015-01-12,613.0,384.0,88.0,270.0,542.0,141.0,135,96.0,2015-01-12


,kirisimalag1,kirisimalag2,kirisimalag3,kirisimalag4,kirisimalag5,kirisimalag6,kirisimalag7,y,date
datetime,,,,,,,,,
2015-01-08,16559.0,7856.0,8112.0,12993.0,27027.0,29856.0,35352,8404.0,2015-01-08
2015-01-09,8404.0,16559.0,7856.0,8112.0,12993.0,27027.0,29856,10997.0,2015-01-09
2015-01-10,10997.0,8404.0,16559.0,7856.0,8112.0,12993.0,27027,28202.0,2015-01-10
2015-01-11,28202.0,10997.0,8404.0,16559.0,7856.0,8112.0,12993,26121.0,2015-01-11
2015-01-12,26121.0,28202.0,10997.0,8404.0,16559.0,7856.0,8112,15286.0,2015-01-12


,keramalag1,keramalag2,keramalag3,keramalag4,keramalag5,keramalag6,keramalag7,y,date
datetime,,,,,,,,,
2015-01-08,0.0,0.0,0.0,75.0,126.0,657.0,151,25.0,2015-01-08
2015-01-09,25.0,0.0,0.0,0.0,75.0,126.0,657,0.0,2015-01-09
2015-01-10,0.0,25.0,0.0,0.0,0.0,75.0,126,510.0,2015-01-10
2015-01-11,510.0,0.0,25.0,0.0,0.0,0.0,75,510.0,2015-01-11
2015-01-12,510.0,510.0,0.0,25.0,0.0,0.0,0,50.0,2015-01-12


,towadalag1,towadalag2,towadalag3,towadalag4,towadalag5,towadalag6,towadalag7,y,date
datetime,,,,,,,,,
2015-01-08,2374.0,2771.0,4732.0,9306.0,17114.0,33795.0,34757,3019.0,2015-01-08
2015-01-09,3019.0,2374.0,2771.0,4732.0,9306.0,17114.0,33795,3865.0,2015-01-09
2015-01-10,3865.0,3019.0,2374.0,2771.0,4732.0,9306.0,17114,16346.0,2015-01-10
2015-01-11,16346.0,3865.0,3019.0,2374.0,2771.0,4732.0,9306,16169.0,2015-01-11
2015-01-12,16169.0,16346.0,3865.0,3019.0,2374.0,2771.0,4732,7301.0,2015-01-12


,nikkolag1,nikkolag2,nikkolag3,nikkolag4,nikkolag5,nikkolag6,nikkolag7,y,date
datetime,,,,,,,,,
2015-01-08,1031.0,3593.0,6571.0,12236.0,22247.0,27011.0,29714,1548.0,2015-01-08
2015-01-09,1548.0,1031.0,3593.0,6571.0,12236.0,22247.0,27011,1216.0,2015-01-09
2015-01-10,1216.0,1548.0,1031.0,3593.0,6571.0,12236.0,22247,11152.0,2015-01-10
2015-01-11,11152.0,1216.0,1548.0,1031.0,3593.0,6571.0,12236,11820.0,2015-01-11
2015-01-12,11820.0,11152.0,1216.0,1548.0,1031.0,3593.0,6571,3454.0,2015-01-12


# weather lag

In [4]:
weather = pd.read_csv("../data/weather.tsv",delimiter="\t")

lag = 7
targets = list(weather["地点"].unique())
columns = ["平均気温(℃)",'最高気温(℃)','最低気温(℃)','降水量の合計(mm)','日照時間(時間)','最深積雪(cm)','降雪量合計(cm)']

for target in targets:
    
    col_names = []
    for col_name in weather.columns:
        col_names.append(target +"_"+ col_name)
    data = weather.loc[weather["地点"]== target,:]
    data.columns = col_names

    feature = pd.DataFrame()
    
    new_columns = []
    for column in columns:
        new_columns.append(target+"_"+column)

    for l in list(range(1,lag+1)):
        for col in new_columns:
            feature.loc[:,col + "lag"+str(l)] = data[col].shift(-(lag-l))
        feature["date"] = pd.to_datetime(pd.read_csv("../data/weather.tsv",delimiter="\t")["年月日"]) + offsets.Day(lag)


    feature.to_csv("../data/"+target+"weather"+"_l"+str(lag)+".csv",index=False,encoding = "shift-jis")
    display(feature.head())

,十和田_平均気温(℃)lag1,十和田_最高気温(℃)lag1,十和田_最低気温(℃)lag1,十和田_降水量の合計(mm)lag1,十和田_日照時間(時間)lag1,十和田_最深積雪(cm)lag1,十和田_降雪量合計(cm)lag1,date,十和田_平均気温(℃)lag2,十和田_最高気温(℃)lag2,十和田_最低気温(℃)lag2,十和田_降水量の合計(mm)lag2,十和田_日照時間(時間)lag2,十和田_最深積雪(cm)lag2,十和田_降雪量合計(cm)lag2,十和田_平均気温(℃)lag3,十和田_最高気温(℃)lag3,十和田_最低気温(℃)lag3,十和田_降水量の合計(mm)lag3,十和田_日照時間(時間)lag3,十和田_最深積雪(cm)lag3,十和田_降雪量合計(cm)lag3,十和田_平均気温(℃)lag4,十和田_最高気温(℃)lag4,十和田_最低気温(℃)lag4,十和田_降水量の合計(mm)lag4,十和田_日照時間(時間)lag4,十和田_最深積雪(cm)lag4,十和田_降雪量合計(cm)lag4,十和田_平均気温(℃)lag5,十和田_最高気温(℃)lag5,十和田_最低気温(℃)lag5,十和田_降水量の合計(mm)lag5,十和田_日照時間(時間)lag5,十和田_最深積雪(cm)lag5,十和田_降雪量合計(cm)lag5,十和田_平均気温(℃)lag6,十和田_最高気温(℃)lag6,十和田_最低気温(℃)lag6,十和田_降水量の合計(mm)lag6,十和田_日照時間(時間)lag6,十和田_最深積雪(cm)lag6,十和田_降雪量合計(cm)lag6,十和田_平均気温(℃)lag7,十和田_最高気温(℃)lag7,十和田_最低気温(℃)lag7,十和田_降水量の合計(mm)lag7,十和田_日照時間(時間)lag7,十和田_最深積雪(cm)lag7,十和田_降雪量合計(cm)lag7
0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2015-01-08,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0,-1.9,0.3,-2.9,5.0,2.1,18.0,22.0
1,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,2015-01-09,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0
2,-0.2,2.4,-3.9,0.5,2.3,13.0,6.0,2015-01-10,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0
3,-0.5,1.8,-2.1,0.0,4.7,10.0,1.0,2015-01-11,-0.2,2.4,-3.9,0.5,2.3,13.0,6.0,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0
4,-1.0,1.7,-5.5,0.0,5.5,10.0,0.0,2015-01-12,-0.5,1.8,-2.1,0.0,4.7,10.0,1.0,-0.2,2.4,-3.9,0.5,2.3,13.0,6.0,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0


,大山_平均気温(℃)lag1,大山_最高気温(℃)lag1,大山_最低気温(℃)lag1,大山_降水量の合計(mm)lag1,大山_日照時間(時間)lag1,大山_最深積雪(cm)lag1,大山_降雪量合計(cm)lag1,date,大山_平均気温(℃)lag2,大山_最高気温(℃)lag2,大山_最低気温(℃)lag2,大山_降水量の合計(mm)lag2,大山_日照時間(時間)lag2,大山_最深積雪(cm)lag2,大山_降雪量合計(cm)lag2,大山_平均気温(℃)lag3,大山_最高気温(℃)lag3,大山_最低気温(℃)lag3,大山_降水量の合計(mm)lag3,大山_日照時間(時間)lag3,大山_最深積雪(cm)lag3,大山_降雪量合計(cm)lag3,大山_平均気温(℃)lag4,大山_最高気温(℃)lag4,大山_最低気温(℃)lag4,大山_降水量の合計(mm)lag4,大山_日照時間(時間)lag4,大山_最深積雪(cm)lag4,大山_降雪量合計(cm)lag4,大山_平均気温(℃)lag5,大山_最高気温(℃)lag5,大山_最低気温(℃)lag5,大山_降水量の合計(mm)lag5,大山_日照時間(時間)lag5,大山_最深積雪(cm)lag5,大山_降雪量合計(cm)lag5,大山_平均気温(℃)lag6,大山_最高気温(℃)lag6,大山_最低気温(℃)lag6,大山_降水量の合計(mm)lag6,大山_日照時間(時間)lag6,大山_最深積雪(cm)lag6,大山_降雪量合計(cm)lag6,大山_平均気温(℃)lag7,大山_最高気温(℃)lag7,大山_最低気温(℃)lag7,大山_降水量の合計(mm)lag7,大山_日照時間(時間)lag7,大山_最深積雪(cm)lag7,大山_降雪量合計(cm)lag7
1095,NaN,NaN,NaN,21.0,NaN,87.0,16.0,2015-01-08,NaN,NaN,NaN,16.0,NaN,93.0,1.0,NaN,NaN,NaN,0.0,NaN,99.0,0.0,NaN,NaN,NaN,0.0,NaN,107.0,0.0,NaN,NaN,NaN,8.0,NaN,119.0,5.0,NaN,NaN,NaN,16.0,NaN,114.0,21.0,NaN,NaN,NaN,30.0,NaN,101.0,42.0
1096,NaN,NaN,NaN,7.5,NaN,93.0,7.0,2015-01-09,NaN,NaN,NaN,21.0,NaN,87.0,16.0,NaN,NaN,NaN,16.0,NaN,93.0,1.0,NaN,NaN,NaN,0.0,NaN,99.0,0.0,NaN,NaN,NaN,0.0,NaN,107.0,0.0,NaN,NaN,NaN,8.0,NaN,119.0,5.0,NaN,NaN,NaN,16.0,NaN,114.0,21.0
1097,NaN,NaN,NaN,12.0,NaN,101.0,10.0,2015-01-10,NaN,NaN,NaN,7.5,NaN,93.0,7.0,NaN,NaN,NaN,21.0,NaN,87.0,16.0,NaN,NaN,NaN,16.0,NaN,93.0,1.0,NaN,NaN,NaN,0.0,NaN,99.0,0.0,NaN,NaN,NaN,0.0,NaN,107.0,0.0,NaN,NaN,NaN,8.0,NaN,119.0,5.0
1098,NaN,NaN,NaN,8.5,NaN,110.0,11.0,2015-01-11,NaN,NaN,NaN,12.0,NaN,101.0,10.0,NaN,NaN,NaN,7.5,NaN,93.0,7.0,NaN,NaN,NaN,21.0,NaN,87.0,16.0,NaN,NaN,NaN,16.0,NaN,93.0,1.0,NaN,NaN,NaN,0.0,NaN,99.0,0.0,NaN,NaN,NaN,0.0,NaN,107.0,0.0
1099,NaN,NaN,NaN,26.5,NaN,137.0,33.0,2015-01-12,NaN,NaN,NaN,8.5,NaN,110.0,11.0,NaN,NaN,NaN,12.0,NaN,101.0,10.0,NaN,NaN,NaN,7.5,NaN,93.0,7.0,NaN,NaN,NaN,21.0,NaN,87.0,16.0,NaN,NaN,NaN,16.0,NaN,93.0,1.0,NaN,NaN,NaN,0.0,NaN,99.0,0.0


,大田_平均気温(℃)lag1,大田_最高気温(℃)lag1,大田_最低気温(℃)lag1,大田_降水量の合計(mm)lag1,大田_日照時間(時間)lag1,大田_最深積雪(cm)lag1,大田_降雪量合計(cm)lag1,date,大田_平均気温(℃)lag2,大田_最高気温(℃)lag2,大田_最低気温(℃)lag2,大田_降水量の合計(mm)lag2,大田_日照時間(時間)lag2,大田_最深積雪(cm)lag2,大田_降雪量合計(cm)lag2,大田_平均気温(℃)lag3,大田_最高気温(℃)lag3,大田_最低気温(℃)lag3,大田_降水量の合計(mm)lag3,大田_日照時間(時間)lag3,大田_最深積雪(cm)lag3,大田_降雪量合計(cm)lag3,大田_平均気温(℃)lag4,大田_最高気温(℃)lag4,大田_最低気温(℃)lag4,大田_降水量の合計(mm)lag4,大田_日照時間(時間)lag4,大田_最深積雪(cm)lag4,大田_降雪量合計(cm)lag4,大田_平均気温(℃)lag5,大田_最高気温(℃)lag5,大田_最低気温(℃)lag5,大田_降水量の合計(mm)lag5,大田_日照時間(時間)lag5,大田_最深積雪(cm)lag5,大田_降雪量合計(cm)lag5,大田_平均気温(℃)lag6,大田_最高気温(℃)lag6,大田_最低気温(℃)lag6,大田_降水量の合計(mm)lag6,大田_日照時間(時間)lag6,大田_最深積雪(cm)lag6,大田_降雪量合計(cm)lag6,大田_平均気温(℃)lag7,大田_最高気温(℃)lag7,大田_最低気温(℃)lag7,大田_降水量の合計(mm)lag7,大田_日照時間(時間)lag7,大田_最深積雪(cm)lag7,大田_降雪量合計(cm)lag7
2190,5.5,7.3,2.6,1.0,0.5,NaN,NaN,2015-01-08,9.0,11.4,6.0,10.0,0.0,NaN,NaN,7.2,13.1,1.5,0.0,6.5,NaN,NaN,7.7,11.6,4.1,0.0,3.2,NaN,NaN,4.0,8.7,0.4,1.0,2.1,NaN,NaN,2.1,3.3,0.1,2.0,0.5,NaN,NaN,1.1,5.0,-1.5,7.5,0.0,NaN,NaN
2191,5.3,7.2,2.8,0.5,0.3,NaN,NaN,2015-01-09,5.5,7.3,2.6,1.0,0.5,NaN,NaN,9.0,11.4,6.0,10.0,0.0,NaN,NaN,7.2,13.1,1.5,0.0,6.5,NaN,NaN,7.7,11.6,4.1,0.0,3.2,NaN,NaN,4.0,8.7,0.4,1.0,2.1,NaN,NaN,2.1,3.3,0.1,2.0,0.5,NaN,NaN
2192,4.1,8.8,-1.2,0.0,0.9,NaN,NaN,2015-01-10,5.3,7.2,2.8,0.5,0.3,NaN,NaN,5.5,7.3,2.6,1.0,0.5,NaN,NaN,9.0,11.4,6.0,10.0,0.0,NaN,NaN,7.2,13.1,1.5,0.0,6.5,NaN,NaN,7.7,11.6,4.1,0.0,3.2,NaN,NaN,4.0,8.7,0.4,1.0,2.1,NaN,NaN
2193,6.4,8.9,4.5,0.0,2.9,NaN,NaN,2015-01-11,4.1,8.8,-1.2,0.0,0.9,NaN,NaN,5.3,7.2,2.8,0.5,0.3,NaN,NaN,5.5,7.3,2.6,1.0,0.5,NaN,NaN,9.0,11.4,6.0,10.0,0.0,NaN,NaN,7.2,13.1,1.5,0.0,6.5,NaN,NaN,7.7,11.6,4.1,0.0,3.2,NaN,NaN
2194,5.9,10.2,2.1,9.5,1.1,NaN,NaN,2015-01-12,6.4,8.9,4.5,0.0,2.9,NaN,NaN,4.1,8.8,-1.2,0.0,0.9,NaN,NaN,5.3,7.2,2.8,0.5,0.3,NaN,NaN,5.5,7.3,2.6,1.0,0.5,NaN,NaN,9.0,11.4,6.0,10.0,0.0,NaN,NaN,7.2,13.1,1.5,0.0,6.5,NaN,NaN


,日光_平均気温(℃)lag1,日光_最高気温(℃)lag1,日光_最低気温(℃)lag1,日光_降水量の合計(mm)lag1,日光_日照時間(時間)lag1,日光_最深積雪(cm)lag1,日光_降雪量合計(cm)lag1,date,日光_平均気温(℃)lag2,日光_最高気温(℃)lag2,日光_最低気温(℃)lag2,日光_降水量の合計(mm)lag2,日光_日照時間(時間)lag2,日光_最深積雪(cm)lag2,日光_降雪量合計(cm)lag2,日光_平均気温(℃)lag3,日光_最高気温(℃)lag3,日光_最低気温(℃)lag3,日光_降水量の合計(mm)lag3,日光_日照時間(時間)lag3,日光_最深積雪(cm)lag3,日光_降雪量合計(cm)lag3,日光_平均気温(℃)lag4,日光_最高気温(℃)lag4,日光_最低気温(℃)lag4,日光_降水量の合計(mm)lag4,日光_日照時間(時間)lag4,日光_最深積雪(cm)lag4,日光_降雪量合計(cm)lag4,日光_平均気温(℃)lag5,日光_最高気温(℃)lag5,日光_最低気温(℃)lag5,日光_降水量の合計(mm)lag5,日光_日照時間(時間)lag5,日光_最深積雪(cm)lag5,日光_降雪量合計(cm)lag5,日光_平均気温(℃)lag6,日光_最高気温(℃)lag6,日光_最低気温(℃)lag6,日光_降水量の合計(mm)lag6,日光_日照時間(時間)lag6,日光_最深積雪(cm)lag6,日光_降雪量合計(cm)lag6,日光_平均気温(℃)lag7,日光_最高気温(℃)lag7,日光_最低気温(℃)lag7,日光_降水量の合計(mm)lag7,日光_日照時間(時間)lag7,日光_最深積雪(cm)lag7,日光_降雪量合計(cm)lag7
3285,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,2015-01-08,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,-7.4,-2.7,-9.7,0.0,6.3,4.0,3.0
3286,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,2015-01-09,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0
3287,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,2015-01-10,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0
3288,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,2015-01-11,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0
3289,-3.5,0.2,-6.3,3.5,5.7,11.0,12.0,2015-01-12,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0


,渡嘉敷_平均気温(℃)lag1,渡嘉敷_最高気温(℃)lag1,渡嘉敷_最低気温(℃)lag1,渡嘉敷_降水量の合計(mm)lag1,渡嘉敷_日照時間(時間)lag1,渡嘉敷_最深積雪(cm)lag1,渡嘉敷_降雪量合計(cm)lag1,date,渡嘉敷_平均気温(℃)lag2,渡嘉敷_最高気温(℃)lag2,渡嘉敷_最低気温(℃)lag2,渡嘉敷_降水量の合計(mm)lag2,渡嘉敷_日照時間(時間)lag2,渡嘉敷_最深積雪(cm)lag2,渡嘉敷_降雪量合計(cm)lag2,渡嘉敷_平均気温(℃)lag3,渡嘉敷_最高気温(℃)lag3,渡嘉敷_最低気温(℃)lag3,渡嘉敷_降水量の合計(mm)lag3,渡嘉敷_日照時間(時間)lag3,渡嘉敷_最深積雪(cm)lag3,渡嘉敷_降雪量合計(cm)lag3,渡嘉敷_平均気温(℃)lag4,渡嘉敷_最高気温(℃)lag4,渡嘉敷_最低気温(℃)lag4,渡嘉敷_降水量の合計(mm)lag4,渡嘉敷_日照時間(時間)lag4,渡嘉敷_最深積雪(cm)lag4,渡嘉敷_降雪量合計(cm)lag4,渡嘉敷_平均気温(℃)lag5,渡嘉敷_最高気温(℃)lag5,渡嘉敷_最低気温(℃)lag5,渡嘉敷_降水量の合計(mm)lag5,渡嘉敷_日照時間(時間)lag5,渡嘉敷_最深積雪(cm)lag5,渡嘉敷_降雪量合計(cm)lag5,渡嘉敷_平均気温(℃)lag6,渡嘉敷_最高気温(℃)lag6,渡嘉敷_最低気温(℃)lag6,渡嘉敷_降水量の合計(mm)lag6,渡嘉敷_日照時間(時間)lag6,渡嘉敷_最深積雪(cm)lag6,渡嘉敷_降雪量合計(cm)lag6,渡嘉敷_平均気温(℃)lag7,渡嘉敷_最高気温(℃)lag7,渡嘉敷_最低気温(℃)lag7,渡嘉敷_降水量の合計(mm)lag7,渡嘉敷_日照時間(時間)lag7,渡嘉敷_最深積雪(cm)lag7,渡嘉敷_降雪量合計(cm)lag7
4380,14.6,16.4,12.8,0.0,0.0,NaN,NaN,2015-01-08,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN,12.2,13.8,10.9,0.0,0.0,NaN,NaN,12.0,14.1,10.7,0.0,1.1,NaN,NaN
4381,12.9,13.5,12.4,0.0,0.1,NaN,NaN,2015-01-09,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN,12.2,13.8,10.9,0.0,0.0,NaN,NaN
4382,13.6,15.3,12.6,0.0,0.1,NaN,NaN,2015-01-10,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN
4383,14.0,15.9,13.1,0.0,6.4,NaN,NaN,2015-01-11,13.6,15.3,12.6,0.0,0.1,NaN,NaN,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN
4384,14.2,16.1,12.9,0.0,7.9,NaN,NaN,2015-01-12,14.0,15.9,13.1,0.0,6.4,NaN,NaN,13.6,15.3,12.6,0.0,0.1,NaN,NaN,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN


...

,青森_平均気温(℃)lag1,青森_最高気温(℃)lag1,青森_最低気温(℃)lag1,青森_降水量の合計(mm)lag1,青森_日照時間(時間)lag1,青森_最深積雪(cm)lag1,青森_降雪量合計(cm)lag1,date,青森_平均気温(℃)lag2,青森_最高気温(℃)lag2,青森_最低気温(℃)lag2,青森_降水量の合計(mm)lag2,青森_日照時間(時間)lag2,青森_最深積雪(cm)lag2,青森_降雪量合計(cm)lag2,青森_平均気温(℃)lag3,青森_最高気温(℃)lag3,青森_最低気温(℃)lag3,青森_降水量の合計(mm)lag3,青森_日照時間(時間)lag3,青森_最深積雪(cm)lag3,青森_降雪量合計(cm)lag3,青森_平均気温(℃)lag4,青森_最高気温(℃)lag4,青森_最低気温(℃)lag4,青森_降水量の合計(mm)lag4,青森_日照時間(時間)lag4,青森_最深積雪(cm)lag4,青森_降雪量合計(cm)lag4,青森_平均気温(℃)lag5,青森_最高気温(℃)lag5,青森_最低気温(℃)lag5,青森_降水量の合計(mm)lag5,青森_日照時間(時間)lag5,青森_最深積雪(cm)lag5,青森_降雪量合計(cm)lag5,青森_平均気温(℃)lag6,青森_最高気温(℃)lag6,青森_最低気温(℃)lag6,青森_降水量の合計(mm)lag6,青森_日照時間(時間)lag6,青森_最深積雪(cm)lag6,青森_降雪量合計(cm)lag6,青森_平均気温(℃)lag7,青森_最高気温(℃)lag7,青森_最低気温(℃)lag7,青森_降水量の合計(mm)lag7,青森_日照時間(時間)lag7,青森_最深積雪(cm)lag7,青森_降雪量合計(cm)lag7
7665,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,2015-01-08,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0,-1.3,1.0,-2.7,15.0,0.5,88.0,19.0
7666,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,2015-01-09,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0
7667,-0.1,2.2,-1.4,27.0,0.4,115.0,31.0,2015-01-10,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0
7668,-0.9,0.0,-1.8,11.0,0.1,123.0,15.0,2015-01-11,-0.1,2.2,-1.4,27.0,0.4,115.0,31.0,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0
7669,-0.9,0.1,-2.0,1.0,0.0,123.0,3.0,2015-01-12,-0.9,0.0,-1.8,11.0,0.1,123.0,15.0,-0.1,2.2,-1.4,27.0,0.4,115.0,31.0,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0


,高森_平均気温(℃)lag1,高森_最高気温(℃)lag1,高森_最低気温(℃)lag1,高森_降水量の合計(mm)lag1,高森_日照時間(時間)lag1,高森_最深積雪(cm)lag1,高森_降雪量合計(cm)lag1,date,高森_平均気温(℃)lag2,高森_最高気温(℃)lag2,高森_最低気温(℃)lag2,高森_降水量の合計(mm)lag2,高森_日照時間(時間)lag2,高森_最深積雪(cm)lag2,高森_降雪量合計(cm)lag2,高森_平均気温(℃)lag3,高森_最高気温(℃)lag3,高森_最低気温(℃)lag3,高森_降水量の合計(mm)lag3,高森_日照時間(時間)lag3,高森_最深積雪(cm)lag3,高森_降雪量合計(cm)lag3,高森_平均気温(℃)lag4,高森_最高気温(℃)lag4,高森_最低気温(℃)lag4,高森_降水量の合計(mm)lag4,高森_日照時間(時間)lag4,高森_最深積雪(cm)lag4,高森_降雪量合計(cm)lag4,高森_平均気温(℃)lag5,高森_最高気温(℃)lag5,高森_最低気温(℃)lag5,高森_降水量の合計(mm)lag5,高森_日照時間(時間)lag5,高森_最深積雪(cm)lag5,高森_降雪量合計(cm)lag5,高森_平均気温(℃)lag6,高森_最高気温(℃)lag6,高森_最低気温(℃)lag6,高森_降水量の合計(mm)lag6,高森_日照時間(時間)lag6,高森_最深積雪(cm)lag6,高森_降雪量合計(cm)lag6,高森_平均気温(℃)lag7,高森_最高気温(℃)lag7,高森_最低気温(℃)lag7,高森_降水量の合計(mm)lag7,高森_日照時間(時間)lag7,高森_最深積雪(cm)lag7,高森_降雪量合計(cm)lag7
8760,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,2015-01-08,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN,-1.6,-0.1,-3.9,0.5,0.1,NaN,NaN,-2.6,0.8,-5.2,0.0,1.1,NaN,NaN
8761,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,2015-01-09,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN,-1.6,-0.1,-3.9,0.5,0.1,NaN,NaN
8762,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,2015-01-10,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN
8763,3.0,6.1,1.1,0.0,6.2,NaN,NaN,2015-01-11,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN
8764,3.8,7.0,1.3,0.0,3.3,NaN,NaN,2015-01-12,3.0,6.1,1.1,0.0,6.2,NaN,NaN,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN


,鳥羽_平均気温(℃)lag1,鳥羽_最高気温(℃)lag1,鳥羽_最低気温(℃)lag1,鳥羽_降水量の合計(mm)lag1,鳥羽_日照時間(時間)lag1,鳥羽_最深積雪(cm)lag1,鳥羽_降雪量合計(cm)lag1,date,鳥羽_平均気温(℃)lag2,鳥羽_最高気温(℃)lag2,鳥羽_最低気温(℃)lag2,鳥羽_降水量の合計(mm)lag2,鳥羽_日照時間(時間)lag2,鳥羽_最深積雪(cm)lag2,鳥羽_降雪量合計(cm)lag2,鳥羽_平均気温(℃)lag3,鳥羽_最高気温(℃)lag3,鳥羽_最低気温(℃)lag3,鳥羽_降水量の合計(mm)lag3,鳥羽_日照時間(時間)lag3,鳥羽_最深積雪(cm)lag3,鳥羽_降雪量合計(cm)lag3,鳥羽_平均気温(℃)lag4,鳥羽_最高気温(℃)lag4,鳥羽_最低気温(℃)lag4,鳥羽_降水量の合計(mm)lag4,鳥羽_日照時間(時間)lag4,鳥羽_最深積雪(cm)lag4,鳥羽_降雪量合計(cm)lag4,鳥羽_平均気温(℃)lag5,鳥羽_最高気温(℃)lag5,鳥羽_最低気温(℃)lag5,鳥羽_降水量の合計(mm)lag5,鳥羽_日照時間(時間)lag5,鳥羽_最深積雪(cm)lag5,鳥羽_降雪量合計(cm)lag5,鳥羽_平均気温(℃)lag6,鳥羽_最高気温(℃)lag6,鳥羽_最低気温(℃)lag6,鳥羽_降水量の合計(mm)lag6,鳥羽_日照時間(時間)lag6,鳥羽_最深積雪(cm)lag6,鳥羽_降雪量合計(cm)lag6,鳥羽_平均気温(℃)lag7,鳥羽_最高気温(℃)lag7,鳥羽_最低気温(℃)lag7,鳥羽_降水量の合計(mm)lag7,鳥羽_日照時間(時間)lag7,鳥羽_最深積雪(cm)lag7,鳥羽_降雪量合計(cm)lag7
9855,5.4,7.2,3.2,0.0,0.9,NaN,NaN,2015-01-08,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN,2.3,6.6,-1.9,0.0,4.3,NaN,NaN,3.0,6.7,-0.2,0.0,6.3,NaN,NaN
9856,4.6,6.4,2.8,0.0,4.5,NaN,NaN,2015-01-09,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN,2.3,6.6,-1.9,0.0,4.3,NaN,NaN
9857,5.8,7.7,3.8,0.0,6.1,NaN,NaN,2015-01-10,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN
9858,5.9,8.0,4.7,0.0,1.9,NaN,NaN,2015-01-11,5.8,7.7,3.8,0.0,6.1,NaN,NaN,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN
9859,5.8,10.5,-1.2,0.0,6.7,NaN,NaN,2015-01-12,5.9,8.0,4.7,0.0,1.9,NaN,NaN,5.8,7.7,3.8,0.0,6.1,NaN,NaN,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN


,鹿児島_平均気温(℃)lag1,鹿児島_最高気温(℃)lag1,鹿児島_最低気温(℃)lag1,鹿児島_降水量の合計(mm)lag1,鹿児島_日照時間(時間)lag1,鹿児島_最深積雪(cm)lag1,鹿児島_降雪量合計(cm)lag1,date,鹿児島_平均気温(℃)lag2,鹿児島_最高気温(℃)lag2,鹿児島_最低気温(℃)lag2,鹿児島_降水量の合計(mm)lag2,鹿児島_日照時間(時間)lag2,鹿児島_最深積雪(cm)lag2,鹿児島_降雪量合計(cm)lag2,鹿児島_平均気温(℃)lag3,鹿児島_最高気温(℃)lag3,鹿児島_最低気温(℃)lag3,鹿児島_降水量の合計(mm)lag3,鹿児島_日照時間(時間)lag3,鹿児島_最深積雪(cm)lag3,鹿児島_降雪量合計(cm)lag3,鹿児島_平均気温(℃)lag4,鹿児島_最高気温(℃)lag4,鹿児島_最低気温(℃)lag4,鹿児島_降水量の合計(mm)lag4,鹿児島_日照時間(時間)lag4,鹿児島_最深積雪(cm)lag4,鹿児島_降雪量合計(cm)lag4,鹿児島_平均気温(℃)lag5,鹿児島_最高気温(℃)lag5,鹿児島_最低気温(℃)lag5,鹿児島_降水量の合計(mm)lag5,鹿児島_日照時間(時間)lag5,鹿児島_最深積雪(cm)lag5,鹿児島_降雪量合計(cm)lag5,鹿児島_平均気温(℃)lag6,鹿児島_最高気温(℃)lag6,鹿児島_最低気温(℃)lag6,鹿児島_降水量の合計(mm)lag6,鹿児島_日照時間(時間)lag6,鹿児島_最深積雪(cm)lag6,鹿児島_降雪量合計(cm)lag6,鹿児島_平均気温(℃)lag7,鹿児島_最高気温(℃)lag7,鹿児島_最低気温(℃)lag7,鹿児島_降水量の合計(mm)lag7,鹿児島_日照時間(時間)lag7,鹿児島_最深積雪(cm)lag7,鹿児島_降雪量合計(cm)lag7
10950,7.9,13.2,4.4,0.0,6.2,0.0,0.0,2015-01-08,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0,4.3,6.9,1.6,4.0,1.2,0.0,0.0,3.9,8.2,1.6,3.0,0.4,0.0,0.0
10951,6.2,10.7,3.7,0.0,3.8,0.0,0.0,2015-01-09,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0,4.3,6.9,1.6,4.0,1.2,0.0,0.0
10952,6.9,12.9,1.9,0.0,8.0,0.0,0.0,2015-01-10,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0
10953,8.7,13.4,4.8,0.0,7.6,0.0,0.0,2015-01-11,6.9,12.9,1.9,0.0,8.0,0.0,0.0,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0
10954,9.4,15.0,3.7,0.0,6.3,0.0,0.0,2015-01-12,8.7,13.4,4.8,0.0,7.6,0.0,0.0,6.9,12.9,1.9,0.0,8.0,0.0,0.0,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0


,鹿角_平均気温(℃)lag1,鹿角_最高気温(℃)lag1,鹿角_最低気温(℃)lag1,鹿角_降水量の合計(mm)lag1,鹿角_日照時間(時間)lag1,鹿角_最深積雪(cm)lag1,鹿角_降雪量合計(cm)lag1,date,鹿角_平均気温(℃)lag2,鹿角_最高気温(℃)lag2,鹿角_最低気温(℃)lag2,鹿角_降水量の合計(mm)lag2,鹿角_日照時間(時間)lag2,鹿角_最深積雪(cm)lag2,鹿角_降雪量合計(cm)lag2,鹿角_平均気温(℃)lag3,鹿角_最高気温(℃)lag3,鹿角_最低気温(℃)lag3,鹿角_降水量の合計(mm)lag3,鹿角_日照時間(時間)lag3,鹿角_最深積雪(cm)lag3,鹿角_降雪量合計(cm)lag3,鹿角_平均気温(℃)lag4,鹿角_最高気温(℃)lag4,鹿角_最低気温(℃)lag4,鹿角_降水量の合計(mm)lag4,鹿角_日照時間(時間)lag4,鹿角_最深積雪(cm)lag4,鹿角_降雪量合計(cm)lag4,鹿角_平均気温(℃)lag5,鹿角_最高気温(℃)lag5,鹿角_最低気温(℃)lag5,鹿角_降水量の合計(mm)lag5,鹿角_日照時間(時間)lag5,鹿角_最深積雪(cm)lag5,鹿角_降雪量合計(cm)lag5,鹿角_平均気温(℃)lag6,鹿角_最高気温(℃)lag6,鹿角_最低気温(℃)lag6,鹿角_降水量の合計(mm)lag6,鹿角_日照時間(時間)lag6,鹿角_最深積雪(cm)lag6,鹿角_降雪量合計(cm)lag6,鹿角_平均気温(℃)lag7,鹿角_最高気温(℃)lag7,鹿角_最低気温(℃)lag7,鹿角_降水量の合計(mm)lag7,鹿角_日照時間(時間)lag7,鹿角_最深積雪(cm)lag7,鹿角_降雪量合計(cm)lag7
12045,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,2015-01-08,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0,-3.6,-2.3,-6.4,1.0,0.2,74.0,6.0,-2.9,-1.4,-5.6,2.5,2.2,72.0,6.0
12046,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,2015-01-09,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0,-3.6,-2.3,-6.4,1.0,0.2,74.0,6.0
12047,-1.3,0.1,-2.7,5.5,0.3,87.0,12.0,2015-01-10,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0
12048,-1.8,-0.3,-3.1,2.5,0.3,94.0,13.0,2015-01-11,-1.3,0.1,-2.7,5.5,0.3,87.0,12.0,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0
12049,-2.5,-0.2,-7.2,1.0,2.8,96.0,7.0,2015-01-12,-1.8,-0.3,-3.1,2.5,0.3,94.0,13.0,-1.3,0.1,-2.7,5.5,0.3,87.0,12.0,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0


## 対応する公園と観測所の天気を結合

In [23]:
mapping = {"akan":["釧路"], "towada":["青森","十和田","鹿角"],"nikko":["日光"],"isesima":["鳥羽"],"ooyama":["大山","大田"],"aso":["高森","熊本"],"kirisima":["鹿児島"],"kerama":["渡嘉敷"]}

In [24]:

weather = pd.read_csv("../data/weather.tsv",delimiter="\t")
lag = 7
targets = list(weather["地点"].unique())
weather_dict = {}
for target in targets:
    weather_dict[target] = pd.read_csv("../data/"+target+"weather"+"_l"+str(lag)+".csv",engine='python', encoding="SHIFT-JIS")
    

In [25]:
feature = pd.DataFrame()
feature["date"] = pd.to_datetime(pd.read_csv("../data/akan_visitor_lag7.csv")["date"])
print(feature.shape)
feature.head()

(731, 1)


,date
0,2015-01-08
1,2015-01-09
2,2015-01-10
3,2015-01-11
4,2015-01-12


In [27]:
for park in mapping:
    feature = pd.DataFrame()
    feature["date"] = pd.to_datetime(pd.read_csv("../data/akan_visitor_lag7.csv")["date"])
        
    for weather_loc in mapping[park]:
        
        data = pd.read_csv("../data/"+weather_loc+"weather"+"_l"+str(lag)+".csv",engine='python', encoding="SHIFT-JIS")
        data["date"] = pd.to_datetime(data["date"])
        feature = pd.merge(feature, data,on="date")

    display(feature.head())
    feature.to_csv("../data/"+park+"_weather"+"_lag"+str(lag)+".csv",index=False,encoding="SHIFT-JIS")
    
    print(feature.shape)

,date,釧路_平均気温(℃)lag1,釧路_最高気温(℃)lag1,釧路_最低気温(℃)lag1,釧路_降水量の合計(mm)lag1,釧路_日照時間(時間)lag1,釧路_最深積雪(cm)lag1,釧路_降雪量合計(cm)lag1,釧路_平均気温(℃)lag2,釧路_最高気温(℃)lag2,釧路_最低気温(℃)lag2,釧路_降水量の合計(mm)lag2,釧路_日照時間(時間)lag2,釧路_最深積雪(cm)lag2,釧路_降雪量合計(cm)lag2,釧路_平均気温(℃)lag3,釧路_最高気温(℃)lag3,釧路_最低気温(℃)lag3,釧路_降水量の合計(mm)lag3,釧路_日照時間(時間)lag3,釧路_最深積雪(cm)lag3,釧路_降雪量合計(cm)lag3,釧路_平均気温(℃)lag4,釧路_最高気温(℃)lag4,釧路_最低気温(℃)lag4,釧路_降水量の合計(mm)lag4,釧路_日照時間(時間)lag4,釧路_最深積雪(cm)lag4,釧路_降雪量合計(cm)lag4,釧路_平均気温(℃)lag5,釧路_最高気温(℃)lag5,釧路_最低気温(℃)lag5,釧路_降水量の合計(mm)lag5,釧路_日照時間(時間)lag5,釧路_最深積雪(cm)lag5,釧路_降雪量合計(cm)lag5,釧路_平均気温(℃)lag6,釧路_最高気温(℃)lag6,釧路_最低気温(℃)lag6,釧路_降水量の合計(mm)lag6,釧路_日照時間(時間)lag6,釧路_最深積雪(cm)lag6,釧路_降雪量合計(cm)lag6,釧路_平均気温(℃)lag7,釧路_最高気温(℃)lag7,釧路_最低気温(℃)lag7,釧路_降水量の合計(mm)lag7,釧路_日照時間(時間)lag7,釧路_最深積雪(cm)lag7,釧路_降雪量合計(cm)lag7
0,2015-01-08,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0,-7.1,-0.9,-13.6,0.0,8.8,8.0,0.0,-5.0,-2.6,-8.7,0.0,4.6,8.0,0.0
1,2015-01-09,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0,-7.1,-0.9,-13.6,0.0,8.8,8.0,0.0
2,2015-01-10,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0
3,2015-01-11,-0.9,2.8,-5.0,0.0,8.9,3.0,0.0,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0
4,2015-01-12,0.3,3.0,-1.4,0.0,4.9,2.0,0.0,-0.9,2.8,-5.0,0.0,8.9,3.0,0.0,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0


(731, 50)


,date,青森_平均気温(℃)lag1,青森_最高気温(℃)lag1,青森_最低気温(℃)lag1,青森_降水量の合計(mm)lag1,青森_日照時間(時間)lag1,青森_最深積雪(cm)lag1,青森_降雪量合計(cm)lag1,青森_平均気温(℃)lag2,青森_最高気温(℃)lag2,青森_最低気温(℃)lag2,青森_降水量の合計(mm)lag2,青森_日照時間(時間)lag2,青森_最深積雪(cm)lag2,青森_降雪量合計(cm)lag2,青森_平均気温(℃)lag3,青森_最高気温(℃)lag3,青森_最低気温(℃)lag3,青森_降水量の合計(mm)lag3,青森_日照時間(時間)lag3,青森_最深積雪(cm)lag3,青森_降雪量合計(cm)lag3,青森_平均気温(℃)lag4,青森_最高気温(℃)lag4,青森_最低気温(℃)lag4,青森_降水量の合計(mm)lag4,青森_日照時間(時間)lag4,青森_最深積雪(cm)lag4,青森_降雪量合計(cm)lag4,青森_平均気温(℃)lag5,青森_最高気温(℃)lag5,青森_最低気温(℃)lag5,青森_降水量の合計(mm)lag5,青森_日照時間(時間)lag5,青森_最深積雪(cm)lag5,青森_降雪量合計(cm)lag5,青森_平均気温(℃)lag6,青森_最高気温(℃)lag6,青森_最低気温(℃)lag6,青森_降水量の合計(mm)lag6,青森_日照時間(時間)lag6,青森_最深積雪(cm)lag6,青森_降雪量合計(cm)lag6,青森_平均気温(℃)lag7,青森_最高気温(℃)lag7,青森_最低気温(℃)lag7,青森_降水量の合計(mm)lag7,青森_日照時間(時間)lag7,青森_最深積雪(cm)lag7,青森_降雪量合計(cm)lag7,十和田_平均気温(℃)lag1,十和田_最高気温(℃)lag1,十和田_最低気温(℃)lag1,十和田_降水量の合計(mm)lag1,十和田_日照時間(時間)lag1,十和田_最深積雪(cm)lag1,十和田_降雪量合計(cm)lag1,十和田_平均気温(℃)lag2,十和田_最高気温(℃)lag2,十和田_最低気温(℃)lag2,十和田_降水量の合計(mm)lag2,十和田_日照時間(時間)lag2,十和田_最深積雪(cm)lag2,十和田_降雪量合計(cm)lag2,十和田_平均気温(℃)lag3,十和田_最高気温(℃)lag3,十和田_最低気温(℃)lag3,十和田_降水量の合計(mm)lag3,十和田_日照時間(時間)lag3,十和田_最深積雪(cm)lag3,十和田_降雪量合計(cm)lag3,十和田_平均気温(℃)lag4,十和田_最高気温(℃)lag4,十和田_最低気温(℃)lag4,十和田_降水量の合計(mm)lag4,十和田_日照時間(時間)lag4,十和田_最深積雪(cm)lag4,十和田_降雪量合計(cm)lag4,十和田_平均気温(℃)lag5,十和田_最高気温(℃)lag5,十和田_最低気温(℃)lag5,十和田_降水量の合計(mm)lag5,十和田_日照時間(時間)lag5,十和田_最深積雪(cm)lag5,十和田_降雪量合計(cm)lag5,十和田_平均気温(℃)lag6,十和田_最高気温(℃)lag6,十和田_最低気温(℃)lag6,十和田_降水量の合計(mm)lag6,十和田_日照時間(時間)lag6,十和田_最深積雪(cm)lag6,十和田_降雪量合計(cm)lag6,十和田_平均気温(℃)lag7,十和田_最高気温(℃)lag7,十和田_最低気温(℃)lag7,十和田_降水量の合計(mm)lag7,十和田_日照時間(時間)lag7,十和田_最深積雪(cm)lag7,十和田_降雪量合計(cm)lag7,鹿角_平均気温(℃)lag1,鹿角_最高気温(℃)lag1,鹿角_最低気温(℃)lag1,鹿角_降水量の合計(mm)lag1,鹿角_日照時間(時間)lag1,鹿角_最深積雪(cm)lag1,鹿角_降雪量合計(cm)lag1,鹿角_平均気温(℃)lag2,鹿角_最高気温(℃)lag2,鹿角_最低気温(℃)lag2,鹿角_降水量の合計(mm)lag2,鹿角_日照時間(時間)lag2,鹿角_最深積雪(cm)lag2,鹿角_降雪量合計(cm)lag2,鹿角_平均気温(℃)lag3,鹿角_最高気温(℃)lag3,鹿角_最低気温(℃)lag3,鹿角_降水量の合計(mm)lag3,鹿角_日照時間(時間)lag3,鹿角_最深積雪(cm)lag3,鹿角_降雪量合計(cm)lag3,鹿角_平均気温(℃)lag4,鹿角_最高気温(℃)lag4,鹿角_最低気温(℃)lag4,鹿角_降水量の合計(mm)lag4,鹿角_日照時間(時間)lag4,鹿角_最深積雪(cm)lag4,鹿角_降雪量合計(cm)lag4,鹿角_平均気温(℃)lag5,鹿角_最高気温(℃)lag5,鹿角_最低気温(℃)lag5,鹿角_降水量の合計(mm)lag5,鹿角_日照時間(時間)lag5,鹿角_最深積雪(cm)lag5,鹿角_降雪量合計(cm)lag5,鹿角_平均気温(℃)lag6,鹿角_最高気温(℃)lag6,鹿角_最低気温(℃)lag6,鹿角_降水量の合計(mm)lag6,鹿角_日照時間(時間)lag6,鹿角_最深積雪(cm)lag6,鹿角_降雪量合計(cm)lag6,鹿角_平均気温(℃)lag7,鹿角_最高気温(℃)lag7,鹿角_最低気温(℃)lag7,鹿角_降水量の合計(mm)lag7,鹿角_日照時間(時間)lag7,鹿角_最深積雪(cm)lag7,鹿角_降雪量合計(cm)lag7
0,2015-01-08,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0,-1.3,1.0,-2.7,15.0,0.5,88.0,19.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0,-1.9,0.3,-2.9,5.0,2.1,18.0,22.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0,-3.6,-2.3,-6.4,1.0,0.2,74.0,6.0,-2.9,-1.4,-5.6,2.5,2.2,72.0,6.0
1,2015-01-09,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0,-3.6,-2.3,-6.4,1.0,0.2,74.0,6.0
2,2015-01-10,-0.1,2.2,-1.4,27.0,0.4,115.0,31.0,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-0.2,2.4

(731, 148)


,date,日光_平均気温(℃)lag1,日光_最高気温(℃)lag1,日光_最低気温(℃)lag1,日光_降水量の合計(mm)lag1,日光_日照時間(時間)lag1,日光_最深積雪(cm)lag1,日光_降雪量合計(cm)lag1,日光_平均気温(℃)lag2,日光_最高気温(℃)lag2,日光_最低気温(℃)lag2,日光_降水量の合計(mm)lag2,日光_日照時間(時間)lag2,日光_最深積雪(cm)lag2,日光_降雪量合計(cm)lag2,日光_平均気温(℃)lag3,日光_最高気温(℃)lag3,日光_最低気温(℃)lag3,日光_降水量の合計(mm)lag3,日光_日照時間(時間)lag3,日光_最深積雪(cm)lag3,日光_降雪量合計(cm)lag3,日光_平均気温(℃)lag4,日光_最高気温(℃)lag4,日光_最低気温(℃)lag4,日光_降水量の合計(mm)lag4,日光_日照時間(時間)lag4,日光_最深積雪(cm)lag4,日光_降雪量合計(cm)lag4,日光_平均気温(℃)lag5,日光_最高気温(℃)lag5,日光_最低気温(℃)lag5,日光_降水量の合計(mm)lag5,日光_日照時間(時間)lag5,日光_最深積雪(cm)lag5,日光_降雪量合計(cm)lag5,日光_平均気温(℃)lag6,日光_最高気温(℃)lag6,日光_最低気温(℃)lag6,日光_降水量の合計(mm)lag6,日光_日照時間(時間)lag6,日光_最深積雪(cm)lag6,日光_降雪量合計(cm)lag6,日光_平均気温(℃)lag7,日光_最高気温(℃)lag7,日光_最低気温(℃)lag7,日光_降水量の合計(mm)lag7,日光_日照時間(時間)lag7,日光_最深積雪(cm)lag7,日光_降雪量合計(cm)lag7
0,2015-01-08,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,-7.4,-2.7,-9.7,0.0,6.3,4.0,3.0
1,2015-01-09,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0
2,2015-01-10,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0
3,2015-01-11,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0
4,2015-01-12,-3.5,0.2,-6.3,3.5,5.7,11.0,12.0,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0


...

(731, 99)


,date,鹿児島_平均気温(℃)lag1,鹿児島_最高気温(℃)lag1,鹿児島_最低気温(℃)lag1,鹿児島_降水量の合計(mm)lag1,鹿児島_日照時間(時間)lag1,鹿児島_最深積雪(cm)lag1,鹿児島_降雪量合計(cm)lag1,鹿児島_平均気温(℃)lag2,鹿児島_最高気温(℃)lag2,鹿児島_最低気温(℃)lag2,鹿児島_降水量の合計(mm)lag2,鹿児島_日照時間(時間)lag2,鹿児島_最深積雪(cm)lag2,鹿児島_降雪量合計(cm)lag2,鹿児島_平均気温(℃)lag3,鹿児島_最高気温(℃)lag3,鹿児島_最低気温(℃)lag3,鹿児島_降水量の合計(mm)lag3,鹿児島_日照時間(時間)lag3,鹿児島_最深積雪(cm)lag3,鹿児島_降雪量合計(cm)lag3,鹿児島_平均気温(℃)lag4,鹿児島_最高気温(℃)lag4,鹿児島_最低気温(℃)lag4,鹿児島_降水量の合計(mm)lag4,鹿児島_日照時間(時間)lag4,鹿児島_最深積雪(cm)lag4,鹿児島_降雪量合計(cm)lag4,鹿児島_平均気温(℃)lag5,鹿児島_最高気温(℃)lag5,鹿児島_最低気温(℃)lag5,鹿児島_降水量の合計(mm)lag5,鹿児島_日照時間(時間)lag5,鹿児島_最深積雪(cm)lag5,鹿児島_降雪量合計(cm)lag5,鹿児島_平均気温(℃)lag6,鹿児島_最高気温(℃)lag6,鹿児島_最低気温(℃)lag6,鹿児島_降水量の合計(mm)lag6,鹿児島_日照時間(時間)lag6,鹿児島_最深積雪(cm)lag6,鹿児島_降雪量合計(cm)lag6,鹿児島_平均気温(℃)lag7,鹿児島_最高気温(℃)lag7,鹿児島_最低気温(℃)lag7,鹿児島_降水量の合計(mm)lag7,鹿児島_日照時間(時間)lag7,鹿児島_最深積雪(cm)lag7,鹿児島_降雪量合計(cm)lag7
0,2015-01-08,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0,4.3,6.9,1.6,4.0,1.2,0.0,0.0,3.9,8.2,1.6,3.0,0.4,0.0,0.0
1,2015-01-09,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0,4.3,6.9,1.6,4.0,1.2,0.0,0.0
2,2015-01-10,6.9,12.9,1.9,0.0,8.0,0.0,0.0,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0,5.4,11.3,0.6,0.0,6.5,0.0,0.0
3,2015-01-11,8.7,13.4,4.8,0.0,7.6,0.0,0.0,6.9,12.9,1.9,0.0,8.0,0.0,0.0,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0,9.4,16.5,3.5,0.0,5.1,0.0,0.0
4,2015-01-12,9.4,15.0,3.7,0.0,6.3,0.0,0.0,8.7,13.4,4.8,0.0,7.6,0.0,0.0,6.9,12.9,1.9,0.0,8.0,0.0,0.0,6.2,10.7,3.7,0.0,3.8,0.0,0.0,7.9,13.2,4.4,0.0,6.2,0.0,0.0,12.5,17.8,7.7,5.0,1.3,0.0,0.0,11.0,16.2,6.1,0.0,5.6,0.0,0.0


(731, 50)


,date,渡嘉敷_平均気温(℃)lag1,渡嘉敷_最高気温(℃)lag1,渡嘉敷_最低気温(℃)lag1,渡嘉敷_降水量の合計(mm)lag1,渡嘉敷_日照時間(時間)lag1,渡嘉敷_最深積雪(cm)lag1,渡嘉敷_降雪量合計(cm)lag1,渡嘉敷_平均気温(℃)lag2,渡嘉敷_最高気温(℃)lag2,渡嘉敷_最低気温(℃)lag2,渡嘉敷_降水量の合計(mm)lag2,渡嘉敷_日照時間(時間)lag2,渡嘉敷_最深積雪(cm)lag2,渡嘉敷_降雪量合計(cm)lag2,渡嘉敷_平均気温(℃)lag3,渡嘉敷_最高気温(℃)lag3,渡嘉敷_最低気温(℃)lag3,渡嘉敷_降水量の合計(mm)lag3,渡嘉敷_日照時間(時間)lag3,渡嘉敷_最深積雪(cm)lag3,渡嘉敷_降雪量合計(cm)lag3,渡嘉敷_平均気温(℃)lag4,渡嘉敷_最高気温(℃)lag4,渡嘉敷_最低気温(℃)lag4,渡嘉敷_降水量の合計(mm)lag4,渡嘉敷_日照時間(時間)lag4,渡嘉敷_最深積雪(cm)lag4,渡嘉敷_降雪量合計(cm)lag4,渡嘉敷_平均気温(℃)lag5,渡嘉敷_最高気温(℃)lag5,渡嘉敷_最低気温(℃)lag5,渡嘉敷_降水量の合計(mm)lag5,渡嘉敷_日照時間(時間)lag5,渡嘉敷_最深積雪(cm)lag5,渡嘉敷_降雪量合計(cm)lag5,渡嘉敷_平均気温(℃)lag6,渡嘉敷_最高気温(℃)lag6,渡嘉敷_最低気温(℃)lag6,渡嘉敷_降水量の合計(mm)lag6,渡嘉敷_日照時間(時間)lag6,渡嘉敷_最深積雪(cm)lag6,渡嘉敷_降雪量合計(cm)lag6,渡嘉敷_平均気温(℃)lag7,渡嘉敷_最高気温(℃)lag7,渡嘉敷_最低気温(℃)lag7,渡嘉敷_降水量の合計(mm)lag7,渡嘉敷_日照時間(時間)lag7,渡嘉敷_最深積雪(cm)lag7,渡嘉敷_降雪量合計(cm)lag7
0,2015-01-08,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN,12.2,13.8,10.9,0.0,0.0,NaN,NaN,12.0,14.1,10.7,0.0,1.1,NaN,NaN
1,2015-01-09,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN,12.2,13.8,10.9,0.0,0.0,NaN,NaN
2,2015-01-10,13.6,15.3,12.6,0.0,0.1,NaN,NaN,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN,13.2,15.5,12.0,0.0,3.1,NaN,NaN
3,2015-01-11,14.0,15.9,13.1,0.0,6.4,NaN,NaN,13.6,15.3,12.6,0.0,0.1,NaN,NaN,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN,15.6,18.6,12.8,0.0,9.7,NaN,NaN
4,2015-01-12,14.2,16.1,12.9,0.0,7.9,NaN,NaN,14.0,15.9,13.1,0.0,6.4,NaN,NaN,13.6,15.3,12.6,0.0,0.1,NaN,NaN,12.9,13.5,12.4,0.0,0.1,NaN,NaN,14.6,16.4,12.8,0.0,0.0,NaN,NaN,18.2,20.7,16.4,7.5,0.0,NaN,NaN,17.7,20.2,15.0,0.0,7.8,NaN,NaN


(731, 50)


# holiday

In [28]:
date = pd.to_datetime(pd.read_csv("../data/akan_visitor_lag7.csv")["date"])

In [29]:
public_holiday = []
for i, d in date.iteritems():
    public_holiday.append(jpholiday.is_holiday(datetime.date(d.to_datetime().year,d.to_datetime().month,d.to_datetime().day)))

date_df = pd.DataFrame(date)
p_holiday = pd.DataFrame(public_holiday)
p_holiday.columns = ["public_holiday"]

p_holiday_df = pd.concat((date,p_holiday),axis=1)

/home/watanabe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
weekday = []
for d in date:
    weekday.append(d.weekday())
    
weekday_df = pd.DataFrame(weekday)
weekday_df.columns = ["weekday"]
data = pd.concat((p_holiday_df,weekday_df),axis=1)

In [32]:
data.loc[(data["weekday"]==5),"holiday"] = True
data.loc[(data["weekday"]==6),"holiday"] = True

data = data.fillna(False)
data = data.rename(columns={'年月日': 'date'})
print(data.shape)
data.head()

(731, 4)


,date,public_holiday,weekday,holiday
0,2015-01-08,False,3,False
1,2015-01-09,False,4,False
2,2015-01-10,False,5,True
3,2015-01-11,False,6,True
4,2015-01-12,True,0,False


In [33]:
data.to_csv("../data/holiday.csv", index=False)

# ゴールデンウィークフラグ

In [129]:
date = pd.date_range(start='2015-1-1', end='2016-12-31', freq='D')
date_df = pd.DataFrame(date)
date_df.columns = ["date"]
date_df.head()

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [130]:
gw_2015 = pd.date_range(start='2015-5-2', end='2015-5-9', freq='D')
gw_2015_df = pd.DataFrame(gw_2015)
gw_2015_df.columns = ["date"]
gw_2015_df.loc[:,"gw"] = True


In [131]:
gw_2016 = pd.date_range(start='2016-5-1', end='2016-5-7', freq='D')
gw_2016_df = pd.DataFrame(gw_2016)
gw_2016_df.columns = ["date"]
gw_2016_df.loc[:,"gw"] = True


In [132]:
date_df = pd.merge(date_df, gw_2015_df, on="date", how="left", copy=False)
date_df = pd.merge(date_df, gw_2016_df, on="date", how="left", copy=False)
date_df

,date,gw_x,gw_y
0,2015-01-01,NaN,NaN
1,2015-01-02,NaN,NaN
2,2015-01-03,NaN,NaN
3,2015-01-04,NaN,NaN
4,2015-01-05,NaN,NaN
5,2015-01-06,NaN,NaN
6,2015-01-07,NaN,NaN
7,2015-01-08,NaN,NaN
8,2015-01-09,NaN,NaN
9,2015-01-10,NaN,NaN


In [133]:
date_df = date_df.fillna(False)
date_df

,date,gw_x,gw_y
0,2015-01-01,False,False
1,2015-01-02,False,False
2,2015-01-03,False,False
3,2015-01-04,False,False
4,2015-01-05,False,False
5,2015-01-06,False,False
6,2015-01-07,False,False
7,2015-01-08,False,False
8,2015-01-09,False,False
9,2015-01-10,False,False


In [134]:
date_df.loc[:,"gw"] = (date_df.loc[:,"gw_x"])+(date_df.loc[:,"gw_y"])
date_df

/home/watanabe/anaconda3/lib/python3.6/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


,date,gw_x,gw_y,gw
0,2015-01-01,False,False,False
1,2015-01-02,False,False,False
2,2015-01-03,False,False,False
3,2015-01-04,False,False,False
4,2015-01-05,False,False,False
5,2015-01-06,False,False,False
6,2015-01-07,False,False,False
7,2015-01-08,False,False,False
8,2015-01-09,False,False,False
9,2015-01-10,False,False,False


In [135]:
date_df = date_df.drop("gw_x", axis=1)
date_df = date_df.drop("gw_y", axis=1)


In [137]:
#上のセルそのままだとxgbが受け付けてくれない pandasがいかんぽい
#https://www.amelt.net/imc/programming/mr/5369/

data = date_df["gw"]
# pandasのデータフレームオブジェクトの生成
df = pd.DataFrame(data)
# ラベルエンコーダーの生成とfit,trasform
lbl = preprocessing.LabelEncoder()
lbl.fit(df)
sample = lbl.transform(df)
# ラベルエンコーダーで変換された値
date_df["gw"] = sample
date_df

/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,date,gw
0,2015-01-01,0
1,2015-01-02,0
2,2015-01-03,0
3,2015-01-04,0
4,2015-01-05,0
5,2015-01-06,0
6,2015-01-07,0
7,2015-01-08,0
8,2015-01-09,0
9,2015-01-10,0


In [138]:
date_df.to_csv("../data/gw.csv", index=False)

# 年末年始フラグ

In [139]:
date = pd.date_range(start='2015-1-1', end='2016-12-31', freq='D')
date_df = pd.DataFrame(date)
date_df.columns = ["date"]
date_df.head()

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [140]:
nn_2015 = pd.date_range(start='2015-1-1', end='2015-1-8', freq='D')
nn_2015_df = pd.DataFrame(nn_2015)
nn_2015_df.columns = ["date"]
nn_2015_df.loc[:,"nn"] = True


In [141]:
nn_2016 = pd.date_range(start='2015-12-24', end='2016-1-8', freq='D')
nn_2016_df = pd.DataFrame(nn_2016)
nn_2016_df.columns = ["date"]
nn_2016_df.loc[:,"nn"] = True

In [142]:
nn_2017 = pd.date_range(start='2016-12-24', end='2016-12-31', freq='D')
nn_2017_df = pd.DataFrame(nn_2017)
nn_2017_df.columns = ["date"]
nn_2017_df.loc[:,"nn"] = True

In [143]:
date_df = pd.merge(date_df, nn_2015_df, on="date", how="left")
date_df = pd.merge(date_df, nn_2016_df, on="date", how="left")
date_df = pd.merge(date_df, nn_2017_df, on="date", how="left")
date_df.columns = ["date","nn15","nn16","nn17"]
date_df

,date,nn15,nn16,nn17
0,2015-01-01,True,NaN,NaN
1,2015-01-02,True,NaN,NaN
2,2015-01-03,True,NaN,NaN
3,2015-01-04,True,NaN,NaN
4,2015-01-05,True,NaN,NaN
5,2015-01-06,True,NaN,NaN
6,2015-01-07,True,NaN,NaN
7,2015-01-08,True,NaN,NaN
8,2015-01-09,NaN,NaN,NaN
9,2015-01-10,NaN,NaN,NaN


In [144]:
date_df = date_df.fillna(False)
date_df

,date,nn15,nn16,nn17
0,2015-01-01,True,False,False
1,2015-01-02,True,False,False
2,2015-01-03,True,False,False
3,2015-01-04,True,False,False
4,2015-01-05,True,False,False
5,2015-01-06,True,False,False
6,2015-01-07,True,False,False
7,2015-01-08,True,False,False
8,2015-01-09,False,False,False
9,2015-01-10,False,False,False


In [145]:
date_df.loc[:,"nn"] = (date_df.loc[:,"nn15"])+(date_df.loc[:,"nn16"])+(date_df.loc[:,"nn17"])
date_df

/home/watanabe/anaconda3/lib/python3.6/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


,date,nn15,nn16,nn17,nn
0,2015-01-01,True,False,False,True
1,2015-01-02,True,False,False,True
2,2015-01-03,True,False,False,True
3,2015-01-04,True,False,False,True
4,2015-01-05,True,False,False,True
5,2015-01-06,True,False,False,True
6,2015-01-07,True,False,False,True
7,2015-01-08,True,False,False,True
8,2015-01-09,False,False,False,False
9,2015-01-10,False,False,False,False


In [146]:
date_df = date_df.drop("nn15", axis=1)
date_df = date_df.drop("nn16", axis=1)
date_df = date_df.drop("nn17", axis=1)


In [147]:
#上のセルそのままだとxgbが受け付けてくれない pandasがいかんぽい
#https://www.amelt.net/imc/programming/mr/5369/

data = date_df["nn"]
# pandasのデータフレームオブジェクトの生成
df = pd.DataFrame(data)
# ラベルエンコーダーの生成とfit,trasform
lbl = preprocessing.LabelEncoder()
lbl.fit(df)
sample = lbl.transform(df)
# ラベルエンコーダーで変換された値
date_df["nn"] = sample
date_df

/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,date,nn
0,2015-01-01,1
1,2015-01-02,1
2,2015-01-03,1
3,2015-01-04,1
4,2015-01-05,1
5,2015-01-06,1
6,2015-01-07,1
7,2015-01-08,1
8,2015-01-09,0
9,2015-01-10,0


In [148]:
date_df.to_csv("../data/nn.csv", index=False)

# 夏休みフラグ

In [149]:
date = pd.date_range(start='2015-1-1', end='2016-12-31', freq='D')
date_df = pd.DataFrame(date)
date_df.columns = ["date"]
date_df.head()

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-04
4,2015-01-05


In [150]:
sv_2015 = pd.date_range(start='2015-7-21', end='2015-9-2', freq='D')
sv_2015_df = pd.DataFrame(sv_2015)
sv_2015_df.columns = ["date"]
sv_2015_df.loc[:,"sv"] = True


In [152]:
sv_2016 = pd.date_range(start='2016-7-21', end='2016-9-2', freq='D')
sv_2016_df = pd.DataFrame(sv_2016)
sv_2016_df.columns = ["date"]
sv_2016_df.loc[:,"sv"] = True


In [153]:
date_df = pd.merge(date_df, sv_2015_df, on="date", how="left")
date_df = pd.merge(date_df, sv_2016_df, on="date", how="left")
date_df.columns = ["date","sv15","sv16"]
date_df

,date,sv15,sv16
0,2015-01-01,NaN,NaN
1,2015-01-02,NaN,NaN
2,2015-01-03,NaN,NaN
3,2015-01-04,NaN,NaN
4,2015-01-05,NaN,NaN
5,2015-01-06,NaN,NaN
6,2015-01-07,NaN,NaN
7,2015-01-08,NaN,NaN
8,2015-01-09,NaN,NaN
9,2015-01-10,NaN,NaN


In [154]:
date_df = date_df.fillna(False)


In [155]:
date_df.loc[:,"sv"] = (date_df.loc[:,"sv15"])+(date_df.loc[:,"sv16"])


/home/watanabe/anaconda3/lib/python3.6/site-packages/pandas/computation/expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


In [156]:
date_df = date_df.drop("sv15", axis=1)
date_df = date_df.drop("sv16", axis=1)
date_df.to_csv("../data/sv.csv", index=False)

In [157]:
#上のセルそのままだとxgbが受け付けてくれない pandasがいかんぽい
#https://www.amelt.net/imc/programming/mr/5369/

data = date_df["sv"]
# pandasのデータフレームオブジェクトの生成
df = pd.DataFrame(data)
# ラベルエンコーダーの生成とfit,trasform
lbl = preprocessing.LabelEncoder()
lbl.fit(df)
sample = lbl.transform(df)
# ラベルエンコーダーで変換された値
date_df["sv"] = sample
date_df

/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/watanabe/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,date,sv
0,2015-01-01,0
1,2015-01-02,0
2,2015-01-03,0
3,2015-01-04,0
4,2015-01-05,0
5,2015-01-06,0
6,2015-01-07,0
7,2015-01-08,0
8,2015-01-09,0
9,2015-01-10,0


In [158]:
date_df.to_csv("../data/sv.csv", index=False)

In [159]:
date_df

,date,sv
0,2015-01-01,0
1,2015-01-02,0
2,2015-01-03,0
3,2015-01-04,0
4,2015-01-05,0
5,2015-01-06,0
6,2015-01-07,0
7,2015-01-08,0
8,2015-01-09,0
9,2015-01-10,0


# snskeyword

とりあえず重要な伊勢島、日光、霧島、十和田についてやる

In [58]:
targets = ["isesima","nikko","kirisima","towada"]
key_mapping = {"isesima":"伊勢志摩国立公園","nikko":"日光国立公園","kirisima":"霧島錦江湾国立公園","towada":"十和田八幡平国立公園"}
sns = pd.read_csv("../data/hotlink.tsv",delimiter="\t")

for target in targets:
    bbs = pd.DataFrame(sns.loc[(sns["keyword"] == key_mapping[target])&(sns["domain"] == "bbs"),["datetime","count"]])
    bbs.columns = ["date","bbs"]
    
    blog = pd.DataFrame(sns.loc[(sns["keyword"] == key_mapping[target])&(sns["domain"] == "blog"),["datetime","count"]])
    blog.columns = ["date","blog"]

    twitter = pd.DataFrame(sns.loc[(sns["keyword"] == key_mapping[target])&(sns["domain"] == "twitter_sampling"),["datetime","count"]])
    twitter.columns = ["date","twitter"]
    
    data = pd.merge(bbs, blog, on="date")
    data = pd.merge(data, twitter, on= "date")
    
    data.to_csv("../data/"+target+"_sns.csv",index=False)
    
    
    

In [60]:
lag = 7
cols = ["bbs","blog","twitter"]
for target in targets:
    feature = pd.DataFrame()
    data = pd.read_csv("../data/"+target+"_sns.csv")
    feature["date"] = pd.to_datetime(pd.read_csv("../data/akan_visitor_lag7.csv")["date"])
    for l in list(range(1,lag+1)):

        for col in cols:
            feature.loc[:,col + "lag"+str(l)] = data[col].shift(-(lag-l))
    display(feature.head())
    feature.to_csv("../data/"+target+"_sns.csv", index=False)

,date,bbslag1,bloglag1,twitterlag1,bbslag2,bloglag2,twitterlag2,bbslag3,bloglag3,twitterlag3,bbslag4,bloglag4,twitterlag4,bbslag5,bloglag5,twitterlag5,bbslag6,bloglag6,twitterlag6,bbslag7,bloglag7,twitterlag7
0,2015-01-08,0.0,2.0,4.0,0.0,0.0,2.0,0.0,1.0,2.0,0.0,1.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,0,0,3
1,2015-01-09,0.0,1.0,3.0,0.0,2.0,4.0,0.0,0.0,2.0,0.0,1.0,2.0,0.0,1.0,4.0,0.0,0.0,2.0,0,1,0
2,2015-01-10,0.0,1.0,4.0,0.0,1.0,3.0,0.0,2.0,4.0,0.0,0.0,2.0,0.0,1.0,2.0,0.0,1.0,4.0,0,0,2
3,2015-01-11,0.0,0.0,2.0,0.0,1.0,4.0,0.0,1.0,3.0,0.0,2.0,4.0,0.0,0.0,2.0,0.0,1.0,2.0,0,1,4
4,2015-01-12,0.0,1.0,3.0,0.0,0.0,2.0,0.0,1.0,4.0,0.0,1.0,3.0,0.0,2.0,4.0,0.0,0.0,2.0,0,1,2


,date,bbslag1,bloglag1,twitterlag1,bbslag2,bloglag2,twitterlag2,bbslag3,bloglag3,twitterlag3,bbslag4,bloglag4,twitterlag4,bbslag5,bloglag5,twitterlag5,bbslag6,bloglag6,twitterlag6,bbslag7,bloglag7,twitterlag7
0,2015-01-08,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,3.0,1.0,0.0,2.0,3.0,0.0,0.0,1.0,0,0,0
1,2015-01-09,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,3.0,1.0,0.0,2.0,3.0,0,0,1
2,2015-01-10,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,2.0,0.0,3.0,1.0,0,2,3
3,2015-01-11,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,2.0,0,3,1
4,2015-01-12,0.0,3.0,4.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,3.0,0,1,2


,date,bbslag1,bloglag1,twitterlag1,bbslag2,bloglag2,twitterlag2,bbslag3,bloglag3,twitterlag3,bbslag4,bloglag4,twitterlag4,bbslag5,bloglag5,twitterlag5,bbslag6,bloglag6,twitterlag6,bbslag7,bloglag7,twitterlag7
0,2015-01-08,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0,1,0
1,2015-01-09,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0,1,0
2,2015-01-10,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,2,0
3,2015-01-11,0.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0,1,0
4,2015-01-12,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0,1,0


,date,bbslag1,bloglag1,twitterlag1,bbslag2,bloglag2,twitterlag2,bbslag3,bloglag3,twitterlag3,bbslag4,bloglag4,twitterlag4,bbslag5,bloglag5,twitterlag5,bbslag6,bloglag6,twitterlag6,bbslag7,bloglag7,twitterlag7
0,2015-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0,1
1,2015-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0,0,1
2,2015-01-10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0,1,1
3,2015-01-11,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,1,0
4,2015-01-12,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2


# visitor+weather+holiday+gw+nn+sv

In [61]:
lag = 7
parks = ["aso","akan","isesima","ooyama","kirisima","kerama","towada","nikko"]
sns = ["isesima","kirisima","towada","nikko"]
gw = pd.read_csv("../data/gw.csv")
nn = pd.read_csv("../data/nn.csv")
sv = pd.read_csv("../data/sv.csv")


for park in parks:
    visitor = pd.read_csv("../data/"+park+"_visitor_lag"+str(lag)+".csv",engine='python', encoding="SHIFT-JIS")
    weather = pd.read_csv("../data/"+park+"_weather_lag"+str(lag)+".csv",engine='python', encoding="SHIFT-JIS")
    feature = pd.merge(visitor,weather,on="date",how='left')
    holiday = pd.read_csv("../data/holiday.csv")
    feature = pd.merge(feature,holiday,on="date",how='left')
    feature = pd.merge(feature,gw,on="date",how='left')
    feature = pd.merge(feature,nn,on="date",how='left')
    feature = pd.merge(feature,sv,on="date",how='left')
    if park in sns:
        sns = pd.read_csv("../data/"+target+"_sns.csv")
        feature = pd.merge(feature,sns,on="date",how='left')
    display(feature.head())

    feature.to_csv("../data/"+park+"_feature_lag"+str(lag)+".csv",index=False,encoding="SHIFT-JIS")
    print(feature.shape)


,datetime,asolag1,asolag2,asolag3,asolag4,asolag5,asolag6,asolag7,y,date,高森_平均気温(℃)lag1,高森_最高気温(℃)lag1,高森_最低気温(℃)lag1,高森_降水量の合計(mm)lag1,高森_日照時間(時間)lag1,高森_最深積雪(cm)lag1,高森_降雪量合計(cm)lag1,高森_平均気温(℃)lag2,高森_最高気温(℃)lag2,高森_最低気温(℃)lag2,高森_降水量の合計(mm)lag2,高森_日照時間(時間)lag2,高森_最深積雪(cm)lag2,高森_降雪量合計(cm)lag2,高森_平均気温(℃)lag3,高森_最高気温(℃)lag3,高森_最低気温(℃)lag3,高森_降水量の合計(mm)lag3,高森_日照時間(時間)lag3,高森_最深積雪(cm)lag3,高森_降雪量合計(cm)lag3,高森_平均気温(℃)lag4,高森_最高気温(℃)lag4,高森_最低気温(℃)lag4,高森_降水量の合計(mm)lag4,高森_日照時間(時間)lag4,高森_最深積雪(cm)lag4,高森_降雪量合計(cm)lag4,高森_平均気温(℃)lag5,高森_最高気温(℃)lag5,高森_最低気温(℃)lag5,高森_降水量の合計(mm)lag5,高森_日照時間(時間)lag5,高森_最深積雪(cm)lag5,高森_降雪量合計(cm)lag5,高森_平均気温(℃)lag6,高森_最高気温(℃)lag6,高森_最低気温(℃)lag6,高森_降水量の合計(mm)lag6,高森_日照時間(時間)lag6,高森_最深積雪(cm)lag6,高森_降雪量合計(cm)lag6,高森_平均気温(℃)lag7,高森_最高気温(℃)lag7,高森_最低気温(℃)lag7,高森_降水量の合計(mm)lag7,高森_日照時間(時間)lag7,高森_最深積雪(cm)lag7,高森_降雪量合計(cm)lag7,熊本_平均気温(℃)lag1,熊本_最高気温(℃)lag1,熊本_最低気温(℃)lag1,熊本_降水量の合計(mm)lag1,熊本_日照時間(時間)lag1,熊本_最深積雪(cm)lag1,熊本_降雪量合計(cm)lag1,熊本_平均気温(℃)lag2,熊本_最高気温(℃)lag2,熊本_最低気温(℃)lag2,熊本_降水量の合計(mm)lag2,熊本_日照時間(時間)lag2,熊本_最深積雪(cm)lag2,熊本_降雪量合計(cm)lag2,熊本_平均気温(℃)lag3,熊本_最高気温(℃)lag3,熊本_最低気温(℃)lag3,熊本_降水量の合計(mm)lag3,熊本_日照時間(時間)lag3,熊本_最深積雪(cm)lag3,熊本_降雪量合計(cm)lag3,熊本_平均気温(℃)lag4,熊本_最高気温(℃)lag4,熊本_最低気温(℃)lag4,熊本_降水量の合計(mm)lag4,熊本_日照時間(時間)lag4,熊本_最深積雪(cm)lag4,熊本_降雪量合計(cm)lag4,熊本_平均気温(℃)lag5,熊本_最高気温(℃)lag5,熊本_最低気温(℃)lag5,熊本_降水量の合計(mm)lag5,熊本_日照時間(時間)lag5,熊本_最深積雪(cm)lag5,熊本_降雪量合計(cm)lag5,熊本_平均気温(℃)lag6,熊本_最高気温(℃)lag6,熊本_最低気温(℃)lag6,熊本_降水量の合計(mm)lag6,熊本_日照時間(時間)lag6,熊本_最深積雪(cm)lag6,熊本_降雪量合計(cm)lag6,熊本_平均気温(℃)lag7,熊本_最高気温(℃)lag7,熊本_最低気温(℃)lag7,熊本_降水量の合計(mm)lag7,熊本_日照時間(時間)lag7,熊本_最深積雪(cm)lag7,熊本_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv
0,2015-01-08,25.0,0.0,100.0,324.0,967.0,1643.0,1369,0.0,2015-01-08,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN,-1.6,-0.1,-3.9,0.5,0.1,NaN,NaN,-2.6,0.8,-5.2,0.0,1.1,NaN,NaN,5.3,10.8,1.1,0.0,9.4,0.0,0.0,11.1,15.2,4.9,7.5,1.6,0.0,0.0,8.8,14.7,3.2,0.0,5.0,0.0,0.0,7.9,15.1,1.0,0.0,5.2,0.0,0.0,2.2,8.2,-2.8,0.0,8.2,0.0,0.0,2.2,5.6,-1.8,0.5,1.7,0.0,0.0,2.2,5.9,-1.3,0.0,6.6,0.0,0.0,False,3,False,0.0,1.0,0.0
1,2015-01-09,0.0,25.0,0.0,100.0,324.0,967.0,1643,50.0,2015-01-09,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN,-1.6,-0.1,-3.9,0.5,0.1,NaN,NaN,4.4,9.1,0.7,0.0,7.3,0.0,0.0,5.3,10.8,1.1,0.0,9.4,0.0,0.0,11.1,15.2,4.9,7.5,1.6,0.0,0.0,8.8,14.7,3.2,0.0,5.0,0.0,0.0,7.9,15.1,1.0,0.0,5.2,0.0,0.0,2.2,8.2,-2.8,0.0,8.2,0.0,0.0,2.2,5.6,-1.8,0.5,1.7,0.0,0.0,False,4,False,0.0,0.0,0.0
2,2015-01-10,50.0,0.0,25.0,0.0,100.0,324.0,967,448.0,2015-01-10,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,-1.7,4.0,-8.4,0.0,8.3,NaN,NaN,4.5,10.4,-1.8,0.0,8.2,0.0,0.0,4.4,9.1,0.7,0.0,7.3,0.0,0.0,5.3,10.8,1.1,0.0,9.4,0.0,0.0,11.1,15.2,4.9,7.5,1.6,0.0,0.0,8.8,14.7,3.2,0.0,5.0,0.0,0.0,7.9,15.1,1.0,0.0,5.2,0.0,0.0,2.2,8.2,-2.8,0.0,8.2,0.0,0.0,False,5,True,0.0,0.0,0.0
3,2015-01-11,448.0,50.0,0.0,25.0,0.0,100.0,324,174.0,2015-01-11,3.0,6.1,1.1,0.0,6.2,NaN,NaN,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,5.7,9.8,-1.2,0.0,2.8,NaN,NaN,7.2,10.9,3.2,0.0,5.4,0.0,0.0,4.5,10.4,-1.8,0.0,8.2,0.0,0.0,4.4,9.1,0.7,0.0,7.3,0.0,0.0,5.3,10.8,1.1,0.0,9.4,0.0,0.0,11.1,15.2,4.9,7.5,1.6,0.0,0.0,8.8,14.7,3.2,0.0,5.0,0.0,0.0,7.9,15.1,1.0,0.0,5.2,0.0,0.0,False,6,True,0.0,0.0,0.0
4,2015-01-12,174.0,448.0,50.0,0.0,25.0,0.0,100,0.0,2015-01-12,3.8,7.0,1.3,0.0,3.3,NaN,NaN,3.0,6.1,1.1,0.0,6.2,NaN,NaN,1.3,5.9,-3.4,0.0,7.5,NaN,NaN,0.5,4.3,-2.1,0.0,7.1,NaN,NaN,1.8,5.9,-1.4,0.0,8.5,NaN,NaN,7.8,12.7,1.5,18.5,0.0,NaN,NaN,6.8,13.5,0.5,0.0,4.8,NaN,NaN,6.6,11.8,0.8,0.0,3.9,0.0,0.0,7.2,10.9,3.2,0.0,5.4,0.0,0

(731, 114)


,datetime,akanlag1,akanlag2,akanlag3,akanlag4,akanlag5,akanlag6,akanlag7,y,date,釧路_平均気温(℃)lag1,釧路_最高気温(℃)lag1,釧路_最低気温(℃)lag1,釧路_降水量の合計(mm)lag1,釧路_日照時間(時間)lag1,釧路_最深積雪(cm)lag1,釧路_降雪量合計(cm)lag1,釧路_平均気温(℃)lag2,釧路_最高気温(℃)lag2,釧路_最低気温(℃)lag2,釧路_降水量の合計(mm)lag2,釧路_日照時間(時間)lag2,釧路_最深積雪(cm)lag2,釧路_降雪量合計(cm)lag2,釧路_平均気温(℃)lag3,釧路_最高気温(℃)lag3,釧路_最低気温(℃)lag3,釧路_降水量の合計(mm)lag3,釧路_日照時間(時間)lag3,釧路_最深積雪(cm)lag3,釧路_降雪量合計(cm)lag3,釧路_平均気温(℃)lag4,釧路_最高気温(℃)lag4,釧路_最低気温(℃)lag4,釧路_降水量の合計(mm)lag4,釧路_日照時間(時間)lag4,釧路_最深積雪(cm)lag4,釧路_降雪量合計(cm)lag4,釧路_平均気温(℃)lag5,釧路_最高気温(℃)lag5,釧路_最低気温(℃)lag5,釧路_降水量の合計(mm)lag5,釧路_日照時間(時間)lag5,釧路_最深積雪(cm)lag5,釧路_降雪量合計(cm)lag5,釧路_平均気温(℃)lag6,釧路_最高気温(℃)lag6,釧路_最低気温(℃)lag6,釧路_降水量の合計(mm)lag6,釧路_日照時間(時間)lag6,釧路_最深積雪(cm)lag6,釧路_降雪量合計(cm)lag6,釧路_平均気温(℃)lag7,釧路_最高気温(℃)lag7,釧路_最低気温(℃)lag7,釧路_降水量の合計(mm)lag7,釧路_日照時間(時間)lag7,釧路_最深積雪(cm)lag7,釧路_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv
0,2015-01-08,4569.0,4162.0,4877.0,6732.0,12343.0,11153.0,11028,2715.0,2015-01-08,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0,-7.1,-0.9,-13.6,0.0,8.8,8.0,0.0,-5.0,-2.6,-8.7,0.0,4.6,8.0,0.0,False,3,False,0.0,1.0,0.0
1,2015-01-09,2715.0,4569.0,4162.0,4877.0,6732.0,12343.0,11153,2258.0,2015-01-09,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0,-7.1,-0.9,-13.6,0.0,8.8,8.0,0.0,False,4,False,0.0,0.0,0.0
2,2015-01-10,2258.0,2715.0,4569.0,4162.0,4877.0,6732.0,12343,4273.0,2015-01-10,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,-8.9,-2.0,-15.4,0.0,8.7,8.0,0.0,False,5,True,0.0,0.0,0.0
3,2015-01-11,4273.0,2258.0,2715.0,4569.0,4162.0,4877.0,6732,4273.0,2015-01-11,-0.9,2.8,-5.0,0.0,8.9,3.0,0.0,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,-9.7,-1.9,-16.4,0.0,8.8,8.0,0.0,False,6,True,0.0,0.0,0.0
4,2015-01-12,4273.0,4273.0,2258.0,2715.0,4569.0,4162.0,4877,2686.0,2015-01-12,0.3,3.0,-1.4,0.0,4.9,2.0,0.0,-0.9,2.8,-5.0,0.0,8.9,3.0,0.0,0.5,3.1,-2.9,0.0,4.4,3.0,0.0,1.6,4.6,-1.1,0.0,7.2,3.0,1.0,-2.6,2.3,-5.6,0.0,7.1,3.0,0.0,-0.9,4.8,-10.1,5.0,0.0,8.0,0.0,-7.5,-0.4,-14.8,0.0,7.5,8.0,0.0,True,0,False,0.0,0.0,0.0


(731, 65)


,datetime,isesimalag1,isesimalag2,isesimalag3,isesimalag4,isesimalag5,isesimalag6,isesimalag7,y,date,鳥羽_平均気温(℃)lag1,鳥羽_最高気温(℃)lag1,鳥羽_最低気温(℃)lag1,鳥羽_降水量の合計(mm)lag1,鳥羽_日照時間(時間)lag1,鳥羽_最深積雪(cm)lag1,鳥羽_降雪量合計(cm)lag1,鳥羽_平均気温(℃)lag2,鳥羽_最高気温(℃)lag2,鳥羽_最低気温(℃)lag2,鳥羽_降水量の合計(mm)lag2,鳥羽_日照時間(時間)lag2,鳥羽_最深積雪(cm)lag2,鳥羽_降雪量合計(cm)lag2,鳥羽_平均気温(℃)lag3,鳥羽_最高気温(℃)lag3,鳥羽_最低気温(℃)lag3,鳥羽_降水量の合計(mm)lag3,鳥羽_日照時間(時間)lag3,鳥羽_最深積雪(cm)lag3,鳥羽_降雪量合計(cm)lag3,鳥羽_平均気温(℃)lag4,鳥羽_最高気温(℃)lag4,鳥羽_最低気温(℃)lag4,鳥羽_降水量の合計(mm)lag4,鳥羽_日照時間(時間)lag4,鳥羽_最深積雪(cm)lag4,鳥羽_降雪量合計(cm)lag4,鳥羽_平均気温(℃)lag5,鳥羽_最高気温(℃)lag5,鳥羽_最低気温(℃)lag5,鳥羽_降水量の合計(mm)lag5,鳥羽_日照時間(時間)lag5,鳥羽_最深積雪(cm)lag5,鳥羽_降雪量合計(cm)lag5,鳥羽_平均気温(℃)lag6,鳥羽_最高気温(℃)lag6,鳥羽_最低気温(℃)lag6,鳥羽_降水量の合計(mm)lag6,鳥羽_日照時間(時間)lag6,鳥羽_最深積雪(cm)lag6,鳥羽_降雪量合計(cm)lag6,鳥羽_平均気温(℃)lag7,鳥羽_最高気温(℃)lag7,鳥羽_最低気温(℃)lag7,鳥羽_降水量の合計(mm)lag7,鳥羽_日照時間(時間)lag7,鳥羽_最深積雪(cm)lag7,鳥羽_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv,bbslag1,bloglag1,twitterlag1,bbslag2,bloglag2,twitterlag2,bbslag3,bloglag3,twitterlag3,bbslag4,bloglag4,twitterlag4,bbslag5,bloglag5,twitterlag5,bbslag6,bloglag6,twitterlag6,bbslag7,bloglag7,twitterlag7
0,2015-01-08,4612.0,9118.0,17740.0,34960.0,45368.0,47035.0,42652,3489.0,2015-01-08,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN,2.3,6.6,-1.9,0.0,4.3,NaN,NaN,3.0,6.7,-0.2,0.0,6.3,NaN,NaN,False,3,False,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0,1
1,2015-01-09,3489.0,4612.0,9118.0,17740.0,34960.0,45368.0,47035,5327.0,2015-01-09,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN,2.3,6.6,-1.9,0.0,4.3,NaN,NaN,False,4,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0,0,1
2,2015-01-10,5327.0,3489.0,4612.0,9118.0,17740.0,34960.0,45368,17676.0,2015-01-10,5.8,7.7,3.8,0.0,6.1,NaN,NaN,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,3.0,6.7,-1.4,0.0,6.5,NaN,NaN,False,5,True,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0,1,1
3,2015-01-11,17676.0,5327.0,3489.0,4612.0,9118.0,17740.0,34960,21679.0,2015-01-11,5.9,8.0,4.7,0.0,1.9,NaN,NaN,5.8,7.7,3.8,0.0,6.1,NaN,NaN,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,4.5,10.8,-1.7,0.0,6.6,NaN,NaN,False,6,True,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,1,0
4,2015-01-12,21679.0,17676.0,5327.0,3489.0,4612.0,9118.0,17740,10215.0,2015-01-12,5.8,10.5,-1.2,0.0,6.7,NaN,NaN,5.9,8.0,4.7,0.0,1.9,NaN,NaN,5.8,7.7,3.8,0.0,6.1,NaN,NaN,4.6,6.4,2.8,0.0,4.5,NaN,NaN,5.4,7.2,3.2,0.0,0.9,NaN,NaN,6.9,10.9,3.1,6.5,1.1,NaN,NaN,5.9,10.4,0.6,0.0,8.4,NaN,NaN,True,0,False,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2


...

(731, 65)


,datetime,towadalag1,towadalag2,towadalag3,towadalag4,towadalag5,towadalag6,towadalag7,y,date,青森_平均気温(℃)lag1,青森_最高気温(℃)lag1,青森_最低気温(℃)lag1,青森_降水量の合計(mm)lag1,青森_日照時間(時間)lag1,青森_最深積雪(cm)lag1,青森_降雪量合計(cm)lag1,青森_平均気温(℃)lag2,青森_最高気温(℃)lag2,青森_最低気温(℃)lag2,青森_降水量の合計(mm)lag2,青森_日照時間(時間)lag2,青森_最深積雪(cm)lag2,青森_降雪量合計(cm)lag2,青森_平均気温(℃)lag3,青森_最高気温(℃)lag3,青森_最低気温(℃)lag3,青森_降水量の合計(mm)lag3,青森_日照時間(時間)lag3,青森_最深積雪(cm)lag3,青森_降雪量合計(cm)lag3,青森_平均気温(℃)lag4,青森_最高気温(℃)lag4,青森_最低気温(℃)lag4,青森_降水量の合計(mm)lag4,青森_日照時間(時間)lag4,青森_最深積雪(cm)lag4,青森_降雪量合計(cm)lag4,青森_平均気温(℃)lag5,青森_最高気温(℃)lag5,青森_最低気温(℃)lag5,青森_降水量の合計(mm)lag5,青森_日照時間(時間)lag5,青森_最深積雪(cm)lag5,青森_降雪量合計(cm)lag5,青森_平均気温(℃)lag6,青森_最高気温(℃)lag6,青森_最低気温(℃)lag6,青森_降水量の合計(mm)lag6,青森_日照時間(時間)lag6,青森_最深積雪(cm)lag6,青森_降雪量合計(cm)lag6,青森_平均気温(℃)lag7,青森_最高気温(℃)lag7,青森_最低気温(℃)lag7,青森_降水量の合計(mm)lag7,青森_日照時間(時間)lag7,青森_最深積雪(cm)lag7,青森_降雪量合計(cm)lag7,十和田_平均気温(℃)lag1,十和田_最高気温(℃)lag1,十和田_最低気温(℃)lag1,十和田_降水量の合計(mm)lag1,十和田_日照時間(時間)lag1,十和田_最深積雪(cm)lag1,十和田_降雪量合計(cm)lag1,十和田_平均気温(℃)lag2,十和田_最高気温(℃)lag2,十和田_最低気温(℃)lag2,十和田_降水量の合計(mm)lag2,十和田_日照時間(時間)lag2,十和田_最深積雪(cm)lag2,十和田_降雪量合計(cm)lag2,十和田_平均気温(℃)lag3,十和田_最高気温(℃)lag3,十和田_最低気温(℃)lag3,十和田_降水量の合計(mm)lag3,十和田_日照時間(時間)lag3,十和田_最深積雪(cm)lag3,十和田_降雪量合計(cm)lag3,十和田_平均気温(℃)lag4,十和田_最高気温(℃)lag4,十和田_最低気温(℃)lag4,十和田_降水量の合計(mm)lag4,十和田_日照時間(時間)lag4,十和田_最深積雪(cm)lag4,十和田_降雪量合計(cm)lag4,十和田_平均気温(℃)lag5,十和田_最高気温(℃)lag5,十和田_最低気温(℃)lag5,十和田_降水量の合計(mm)lag5,十和田_日照時間(時間)lag5,十和田_最深積雪(cm)lag5,十和田_降雪量合計(cm)lag5,十和田_平均気温(℃)lag6,十和田_最高気温(℃)lag6,十和田_最低気温(℃)lag6,十和田_降水量の合計(mm)lag6,十和田_日照時間(時間)lag6,十和田_最深積雪(cm)lag6,十和田_降雪量合計(cm)lag6,十和田_平均気温(℃)lag7,十和田_最高気温(℃)lag7,十和田_最低気温(℃)lag7,十和田_降水量の合計(mm)lag7,十和田_日照時間(時間)lag7,十和田_最深積雪(cm)lag7,十和田_降雪量合計(cm)lag7,鹿角_平均気温(℃)lag1,鹿角_最高気温(℃)lag1,鹿角_最低気温(℃)lag1,鹿角_降水量の合計(mm)lag1,鹿角_日照時間(時間)lag1,鹿角_最深積雪(cm)lag1,鹿角_降雪量合計(cm)lag1,鹿角_平均気温(℃)lag2,鹿角_最高気温(℃)lag2,鹿角_最低気温(℃)lag2,鹿角_降水量の合計(mm)lag2,鹿角_日照時間(時間)lag2,鹿角_最深積雪(cm)lag2,鹿角_降雪量合計(cm)lag2,鹿角_平均気温(℃)lag3,鹿角_最高気温(℃)lag3,鹿角_最低気温(℃)lag3,鹿角_降水量の合計(mm)lag3,鹿角_日照時間(時間)lag3,鹿角_最深積雪(cm)lag3,鹿角_降雪量合計(cm)lag3,鹿角_平均気温(℃)lag4,鹿角_最高気温(℃)lag4,鹿角_最低気温(℃)lag4,鹿角_降水量の合計(mm)lag4,鹿角_日照時間(時間)lag4,鹿角_最深積雪(cm)lag4,鹿角_降雪量合計(cm)lag4,鹿角_平均気温(℃)lag5,鹿角_最高気温(℃)lag5,鹿角_最低気温(℃)lag5,鹿角_降水量の合計(mm)lag5,鹿角_日照時間(時間)lag5,鹿角_最深積雪(cm)lag5,鹿角_降雪量合計(cm)lag5,鹿角_平均気温(℃)lag6,鹿角_最高気温(℃)lag6,鹿角_最低気温(℃)lag6,鹿角_降水量の合計(mm)lag6,鹿角_日照時間(時間)lag6,鹿角_最深積雪(cm)lag6,鹿角_降雪量合計(cm)lag6,鹿角_平均気温(℃)lag7,鹿角_最高気温(℃)lag7,鹿角_最低気温(℃)lag7,鹿角_降水量の合計(mm)lag7,鹿角_日照時間(時間)lag7,鹿角_最深積雪(cm)lag7,鹿角_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv
0,2015-01-08,2374.0,2771.0,4732.0,9306.0,17114.0,33795.0,34757,3019.0,2015-01-08,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0,-1.3,1.0,-2.7,15.0,0.5,88.0,19.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0,-1.9,0.3,-2.9,5.0,2.1,18.0,22.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,0.5,90.0,22.0,-3.6,-2.3,-6.4,1.0,0.2,74.0,6.0,-2.9,-1.4,-5.6,2.5,2.2,72.0,6.0,False,3,False,0.0,1.0,0.0
1,2015-01-09,3019.0,2374.0,2771.0,4732.0,9306.0,17114.0,33795,3865.0,2015-01-09,0.1,2.1,-1.6,2.5,0.0,88.0,6.0,-3.9,-0.8,-5.9,8.0,0.2,83.0,8.0,1.3,8.6,-2.9,12.5,0.0,84.0,2.0,2.1,3.6,0.1,1.0,0.0,93.0,0.0,0.9,3.5,-1.8,1.0,2.8,109.0,0.0,-3.3,-1.7,-5.1,10.0,0.2,110.0,15.0,-2.5,-1.2,-3.4,6.5,0.3,99.0,15.0,0.2,2.2,-1.4,0.0,1.8,9.0,4.0,-3.7,-1.2,-5.5,0.0,4.2,11.0,6.0,2.0,7.9,-3.1,0.0,1.2,10.0,0.0,0.8,5.4,-5.1,0.0,4.0,14.0,0.0,0.6,3.4,-1.6,0.0,2.6,19.0,1.0,-4.0,-1.6,-8.9,2.0,1.4,21.0,9.0,-3.9,-1.1,-8.0,0.0,3.3,17.0,0.0,-1.4,-0.3,-3.3,1.0,0.1,82.0,3.0,-5.2,-2.9,-6.4,3.5,0.1,81.0,17.0,-0.8,6.2,-5.0,9.0,0.0,72.0,1.0,-1.2,2.9,-5.2,0.0,0.0,80.0,0.0,-2.4,0.4,-6.5,0.0,1.8,89.0,1.0,-3.9,-1.9,-5.7,6.5,

(731, 163)


,datetime,nikkolag1,nikkolag2,nikkolag3,nikkolag4,nikkolag5,nikkolag6,nikkolag7,y,date,日光_平均気温(℃)lag1,日光_最高気温(℃)lag1,日光_最低気温(℃)lag1,日光_降水量の合計(mm)lag1,日光_日照時間(時間)lag1,日光_最深積雪(cm)lag1,日光_降雪量合計(cm)lag1,日光_平均気温(℃)lag2,日光_最高気温(℃)lag2,日光_最低気温(℃)lag2,日光_降水量の合計(mm)lag2,日光_日照時間(時間)lag2,日光_最深積雪(cm)lag2,日光_降雪量合計(cm)lag2,日光_平均気温(℃)lag3,日光_最高気温(℃)lag3,日光_最低気温(℃)lag3,日光_降水量の合計(mm)lag3,日光_日照時間(時間)lag3,日光_最深積雪(cm)lag3,日光_降雪量合計(cm)lag3,日光_平均気温(℃)lag4,日光_最高気温(℃)lag4,日光_最低気温(℃)lag4,日光_降水量の合計(mm)lag4,日光_日照時間(時間)lag4,日光_最深積雪(cm)lag4,日光_降雪量合計(cm)lag4,日光_平均気温(℃)lag5,日光_最高気温(℃)lag5,日光_最低気温(℃)lag5,日光_降水量の合計(mm)lag5,日光_日照時間(時間)lag5,日光_最深積雪(cm)lag5,日光_降雪量合計(cm)lag5,日光_平均気温(℃)lag6,日光_最高気温(℃)lag6,日光_最低気温(℃)lag6,日光_降水量の合計(mm)lag6,日光_日照時間(時間)lag6,日光_最深積雪(cm)lag6,日光_降雪量合計(cm)lag6,日光_平均気温(℃)lag7,日光_最高気温(℃)lag7,日光_最低気温(℃)lag7,日光_降水量の合計(mm)lag7,日光_日照時間(時間)lag7,日光_最深積雪(cm)lag7,日光_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv
0,2015-01-08,1031.0,3593.0,6571.0,12236.0,22247.0,27011.0,29714,1548.0,2015-01-08,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,-7.4,-2.7,-9.7,0.0,6.3,4.0,3.0,False,3,False,0.0,1.0,0.0
1,2015-01-09,1548.0,1031.0,3593.0,6571.0,12236.0,22247.0,27011,1216.0,2015-01-09,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,False,4,False,0.0,0.0,0.0
2,2015-01-10,1216.0,1548.0,1031.0,3593.0,6571.0,12236.0,22247,11152.0,2015-01-10,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,False,5,True,0.0,0.0,0.0
3,2015-01-11,11152.0,1216.0,1548.0,1031.0,3593.0,6571.0,12236,11820.0,2015-01-11,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,False,6,True,0.0,0.0,0.0
4,2015-01-12,11820.0,11152.0,1216.0,1548.0,1031.0,3593.0,6571,3454.0,2015-01-12,-3.5,0.2,-6.3,3.5,5.7,11.0,12.0,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,True,0,False,0.0,0.0,0.0


(731, 65)


In [117]:
feature.head()

,datetime,nikkolag1,nikkolag2,nikkolag3,nikkolag4,nikkolag5,nikkolag6,nikkolag7,y,date,日光_平均気温(℃)lag1,日光_最高気温(℃)lag1,日光_最低気温(℃)lag1,日光_降水量の合計(mm)lag1,日光_日照時間(時間)lag1,日光_最深積雪(cm)lag1,日光_降雪量合計(cm)lag1,日光_平均気温(℃)lag2,日光_最高気温(℃)lag2,日光_最低気温(℃)lag2,日光_降水量の合計(mm)lag2,日光_日照時間(時間)lag2,日光_最深積雪(cm)lag2,日光_降雪量合計(cm)lag2,日光_平均気温(℃)lag3,日光_最高気温(℃)lag3,日光_最低気温(℃)lag3,日光_降水量の合計(mm)lag3,日光_日照時間(時間)lag3,日光_最深積雪(cm)lag3,日光_降雪量合計(cm)lag3,日光_平均気温(℃)lag4,日光_最高気温(℃)lag4,日光_最低気温(℃)lag4,日光_降水量の合計(mm)lag4,日光_日照時間(時間)lag4,日光_最深積雪(cm)lag4,日光_降雪量合計(cm)lag4,日光_平均気温(℃)lag5,日光_最高気温(℃)lag5,日光_最低気温(℃)lag5,日光_降水量の合計(mm)lag5,日光_日照時間(時間)lag5,日光_最深積雪(cm)lag5,日光_降雪量合計(cm)lag5,日光_平均気温(℃)lag6,日光_最高気温(℃)lag6,日光_最低気温(℃)lag6,日光_降水量の合計(mm)lag6,日光_日照時間(時間)lag6,日光_最深積雪(cm)lag6,日光_降雪量合計(cm)lag6,日光_平均気温(℃)lag7,日光_最高気温(℃)lag7,日光_最低気温(℃)lag7,日光_降水量の合計(mm)lag7,日光_日照時間(時間)lag7,日光_最深積雪(cm)lag7,日光_降雪量合計(cm)lag7,public_holiday,weekday,holiday,gw,nn,sv
0,2015-01-08,1031.0,3593.0,6571.0,12236.0,22247.0,27011.0,29714,1548.0,2015-01-08,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,-7.4,-2.7,-9.7,0.0,6.3,4.0,3.0,False,3,False,False,True,False
1,2015-01-09,1548.0,1031.0,3593.0,6571.0,12236.0,22247.0,27011,1216.0,2015-01-09,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,-7.1,-4.1,-9.1,0.0,4.0,2.0,1.0,False,4,False,False,False,False
2,2015-01-10,1216.0,1548.0,1031.0,3593.0,6571.0,12236.0,22247,11152.0,2015-01-10,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,-7.0,-3.8,-9.6,0.5,6.1,4.0,2.0,False,5,True,False,False,False
3,2015-01-11,11152.0,1216.0,1548.0,1031.0,3593.0,6571.0,12236,11820.0,2015-01-11,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,-2.3,1.7,-8.9,0.0,3.7,0.0,0.0,False,6,True,False,False,False
4,2015-01-12,11820.0,11152.0,1216.0,1548.0,1031.0,3593.0,6571,3454.0,2015-01-12,-3.5,0.2,-6.3,3.5,5.7,11.0,12.0,-4.7,-2.6,-6.3,0.0,5.2,2.0,2.0,-4.1,-1.2,-5.8,0.0,8.6,0.0,0.0,-4.4,-0.7,-6.7,0.0,5.8,0.0,0.0,-5.4,-2.7,-8.9,0.0,7.8,0.0,0.0,0.3,3.9,-4.7,1.5,0.1,1.0,1.0,-1.1,2.5,-5.1,0.0,8.6,1.0,1.0,True,0,False,False,False,False


In [124]:
type(np.array(feature.loc[0,"sv"]))

numpy.ndarray

In [120]:
type(feature.loc[0,"holiday"])

numpy.bool_

In [121]:
feature.loc[0,"holiday"]

False

In [126]:
np.array(feature.loc[0,"sv"])

IndexError: too many indices for array